In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from torch.cuda.amp import autocast, GradScaler

# =============================================================================
# SIMPLE BASELINE: MLP PROJECTION + CLIP LOSS
# =============================================================================

# -----------------------------
# 0. DEVICE
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu_device = torch.device("cpu")
print(f"🚀 Using device: {device}")

# -----------------------------
# 1. DATA LOADING
# -----------------------------
print("📂 Loading data...")

TRAIN_PATH = "/kaggle/input/aml-competition/train/train/train.npz"
TEST_PATH  = "/kaggle/input/aml-competition/test/test/test.clean.npz"

train_data = np.load(TRAIN_PATH)
test_data  = np.load(TEST_PATH)

tx_train = train_data["captions/embeddings"]  # (N_pairs, 1024)
im_train = train_data["images/embeddings"]    # (N_images, 1536)
tx_test  = test_data["captions/embeddings"]   # (N_test, 1024)

print(f"✓ Train captions: {tx_train.shape}")
print(f"✓ Train images:  {im_train.shape}")
print(f"✓ Test captions:  {tx_test.shape}")

# -----------------------------
# 2. PREPROCESSING
# -----------------------------
print("🔧 Preprocessing embeddings...")

tx_train_t         = torch.as_tensor(tx_train, dtype=torch.float32, device=cpu_device)
im_train_t_unique  = torch.as_tensor(im_train, dtype=torch.float32, device=cpu_device)
tx_test_t          = torch.as_tensor(tx_test,  dtype=torch.float32, device=cpu_device)

# Global mean-centering + L2 normalization (text and image)
tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)

tx_train_t        = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t         = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

# Expand images to match captions (5 captions per image)
repeat_factor = tx_train_t.size(0) // im_train_t_unique.size(0)
assert repeat_factor * im_train_t_unique.size(0) == tx_train_t.size(0), "Non-integer repeat factor"
im_train_exp = im_train_t_unique.repeat_interleave(repeat_factor, dim=0)

print(f"✓ Expanded images: {im_train_exp.shape}")

# -----------------------------
# 3. TRAIN / VAL SPLIT (BY IMAGE ID)
# -----------------------------
print("📏 Creating train/val split (by image id)...")

N_pairs    = tx_train_t.size(0)
N_images   = im_train_t_unique.size(0)
val_frac   = 0.1
val_img_n  = max(1, int(val_frac * N_images))

# Image id for each caption-image pair (0..N_images-1)
pair_img_ids = torch.arange(N_pairs, device=cpu_device) // repeat_factor  # (N_pairs,)

# Randomly split images into train/val
perm_imgs     = torch.randperm(N_images, device=cpu_device)
val_img_ids   = perm_imgs[:val_img_n]            # image ids in val
train_img_ids = perm_imgs[val_img_n:]           # image ids in train

# Build boolean mask over pairs based on image ids
is_val_img = torch.zeros(N_images, dtype=torch.bool, device=cpu_device)
is_val_img[val_img_ids] = True
is_val_pair = is_val_img[pair_img_ids]          # (N_pairs,)

val_idx   = torch.nonzero(is_val_pair, as_tuple=False).squeeze(1)
train_idx = torch.nonzero(~is_val_pair, as_tuple=False).squeeze(1)

tx_val_t        = tx_train_t[val_idx]
tx_train_t_sub  = tx_train_t[train_idx]
im_train_exp_sub = im_train_exp[train_idx]

# Build validation gallery: only val images
val_img_ids_sorted, _ = torch.sort(val_img_ids)
im_val = im_train_t_unique[val_img_ids_sorted]   # (N_val_images, 1536)

# Map global image ids -> local indices in val gallery
map_global_to_local = -torch.ones(N_images, dtype=torch.long, device=cpu_device)
map_global_to_local[val_img_ids_sorted] = torch.arange(val_img_ids_sorted.size(0), device=cpu_device)

val_pair_img_global = pair_img_ids[val_idx]                 # global ids for each val pair
val_img_local       = map_global_to_local[val_pair_img_global]  # local indices in val gallery

# Subset for faster per-epoch validation
subset_size         = min(5000, tx_val_t.size(0))
val_query_subset    = tx_val_t[:subset_size]
val_indices_subset  = val_img_local[:subset_size]

print(f"✓ Train pairs: {tx_train_t_sub.size(0)}, Val pairs: {tx_val_t.size(0)}")
print(f"✓ Val images: {im_val.size(0)}")

# -----------------------------
# 4. MODEL
# -----------------------------
class SimpleAlignmentModel(nn.Module):
    """
    Simple baseline: independent MLPs for text and image into shared proj_dim.
    """
    def __init__(self, text_dim=1024, img_dim=1536, proj_dim=1536,
                 hidden_dim=3072, dropout=0.2):
        super().__init__()

        # TEXT PATHWAY
        self.text_proj = nn.Sequential(
            nn.Linear(text_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, proj_dim),
            nn.LayerNorm(proj_dim),
        )

        # IMAGE PATHWAY
        self.img_proj = nn.Sequential(
            nn.Linear(img_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, proj_dim),  # ✅ from hidden_dim -> proj_dim
            nn.LayerNorm(proj_dim),
        )

    def encode_text(self, x: torch.Tensor) -> torch.Tensor:
        z = self.text_proj(x)
        return F.normalize(z, p=2, dim=1)

    def encode_image(self, x: torch.Tensor) -> torch.Tensor:
        z = self.img_proj(x)
        return F.normalize(z, p=2, dim=1)

    def forward(self, text_x: torch.Tensor, img_x: torch.Tensor):
        return self.encode_text(text_x), self.encode_image(img_x)

# -----------------------------
# 5. CLIP-STYLE CONTRASTIVE LOSS
# -----------------------------
class CLIPLoss(nn.Module):
    """
    Symmetric InfoNCE loss used in CLIP:
    - For each text, correct image is on the diagonal.
    - Both text->image and image->text directions.
    """
    def __init__(self, temperature: float = 0.07):
        super().__init__()
        self.temperature = temperature

    def forward(self, z_t: torch.Tensor, z_i: torch.Tensor) -> torch.Tensor:
        # z_t, z_i: [B, D], already L2-normalized
        logits_per_text  = z_t @ z_i.T / self.temperature   # [B, B]
        logits_per_image = z_i @ z_t.T / self.temperature   # [B, B]

        targets = torch.arange(z_t.size(0), device=z_t.device)
        loss_t = F.cross_entropy(logits_per_text, targets)
        loss_i = F.cross_entropy(logits_per_image, targets)
        return (loss_t + loss_i) / 2.0

# -----------------------------
# 6. VALIDATION: RECALL@K
# -----------------------------
@torch.no_grad()
def recall_at_k(
    model: nn.Module,
    tx_queries: torch.Tensor,        # [Nq, text_dim], on CPU
    im_database: torch.Tensor,       # [Nd, img_dim], on CPU
    query_img_indices: torch.Tensor, # [Nq], local indices in [0, Nd)
    chunk_size: int = 1024,
    ks=(1, 5, 10, 50),
):
    model.eval()
    dev = next(model.parameters()).device
    all_correct_in_top_k = {k: [] for k in ks}

    # Encode queries
    Q_embed_list = []
    for i in range(0, tx_queries.size(0), chunk_size):
        chunk = tx_queries[i:i + chunk_size].to(dev)
        Q_embed_list.append(model.encode_text(chunk))
    Q_embed = torch.cat(Q_embed_list, dim=0)  # [Nq, D]

    # Encode database
    D_embed_list = []
    for i in range(0, im_database.size(0), chunk_size):
        chunk = im_database[i:i + chunk_size].to(dev)
        D_embed_list.append(model.encode_image(chunk))
    D_embed = torch.cat(D_embed_list, dim=0)  # [Nd, D]

    gt = query_img_indices.to(dev)

    for i in tqdm(range(0, Q_embed.size(0), chunk_size),
                  desc="Validation", leave=False):
        q_chunk  = Q_embed[i:i + chunk_size]             # [b, D]
        gt_chunk = gt[i:i + chunk_size]                  # [b]

        sim_chunk = q_chunk @ D_embed.T                  # [b, Nd]
        top_indices = torch.argsort(sim_chunk, dim=1, descending=True)

        for k in ks:
            top_k_preds = top_indices[:, :k]             # [b, k]
            correct = (top_k_preds == gt_chunk.unsqueeze(1)).any(dim=1)
            all_correct_in_top_k[k].append(correct)

    recalls = {}
    for k in ks:
        recalls[f"R@{k}"] = torch.cat(all_correct_in_top_k[k]).float().mean().item()
    return recalls

# -----------------------------
# 7. TRAINING SETUP
# -----------------------------
EPOCHS       = 50
BATCH_SIZE   = 512
LR           = 3e-4
WEIGHT_DECAY = 1e-5
DROPOUT      = 0.2

RUN_ID    = "simple_baseline_clip"
SAVE_PATH = f"{RUN_ID}_best.pth"

print("🏗️ Building model...")
model = SimpleAlignmentModel(
    text_dim=tx_train_t.size(1),
    img_dim=im_train_t_unique.size(1),
    proj_dim=im_train_t_unique.size(1),
    hidden_dim=3072,
    dropout=DROPOUT,
).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"📊 Model parameters: {total_params:,}")

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

scaler = GradScaler() if device.type == "cuda" else None
clip_loss_fn = CLIPLoss(temperature=0.07)

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
)

print(f"✓ DataLoader ready: {len(train_dataset)} samples")

# -----------------------------
# 8. TRAINING LOOP
# -----------------------------
print(f"\n{'='*70}")
print(f"🚀 STARTING TRAINING: {RUN_ID}")
print(f"{'='*70}\n")

best_val_r10 = 0.0

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch:02d}/{EPOCHS}", leave=False)

    for tx_batch, im_batch in pbar:
        tx_batch = tx_batch.to(device)
        im_batch = im_batch.to(device)

        optimizer.zero_grad()

        if scaler is not None:
            with autocast():
                z_t, z_i = model(tx_batch, im_batch)
                loss = clip_loss_fn(z_t, z_i)

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
        else:
            z_t, z_i = model(tx_batch, im_batch)
            loss = clip_loss_fn(z_t, z_i)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

    avg_loss = total_loss / len(train_loader)
    current_lr = scheduler.get_last_lr()[0]
    scheduler.step()

    # Per-epoch validation on subset
    rec = recall_at_k(
        model,
        val_query_subset,
        im_val,
        val_indices_subset,
        chunk_size=1024,
    )
    current_r10 = rec['R@10']

    save_marker = ""
    if current_r10 > best_val_r10:
        best_val_r10 = current_r10
        torch.save(model.state_dict(), SAVE_PATH)
        save_marker = " ✨ BEST"

    print(
        f"Epoch {epoch:02d}: "
        f"Loss={avg_loss:.4f} | LR={current_lr:.1e} | "
        f"R@1={rec['R@1']:.4f} R@5={rec['R@5']:.4f} R@10={current_r10:.4f}{save_marker}"
    )

print(f"\n{'='*70}")
print(f"🎉 Training complete! Best R@10 (subset): {best_val_r10:.4f}")
print(f"{'='*70}\n")

# -----------------------------
# 9. INFERENCE ON TEST SET
# -----------------------------
print(f"📥 Loading best model from {SAVE_PATH}...")
model.load_state_dict(torch.load(SAVE_PATH, map_location=device))
model.to(device)
model.eval()

print("🔮 Generating test predictions...")
with torch.no_grad():
    preds_list = []
    for i in tqdm(range(0, tx_test_t.size(0), 1024), desc="Test inference"):
        chunk = tx_test_t[i:i + 1024].to(device)
        preds_list.append(model.encode_text(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)

submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds],
})

submission_filename = f"submission_{RUN_ID}.csv"
submission.to_csv(submission_filename, index=False)
print(f"✅ Submission saved: {submission_filename}")

# -----------------------------
# 10. FULL VALIDATION METRICS (VAL QUERIES VS VAL IMAGES)
# -----------------------------
print("\n📊 Computing full validation metrics on val split...")
rec_full = recall_at_k(
    model,
    tx_val_t,
    im_val,
    val_img_local,
    chunk_size=1024,
)

print(f"\n{'='*70}")
print("🎯 FINAL VALIDATION RESULTS (VAL CAPTIONS → VAL IMAGES)")
print(f"{'='*70}")
print(f"Recall@1:  {rec_full['R@1']:.4f}")
print(f"Recall@5:  {rec_full['R@5']:.4f}")
print(f"Recall@10: {rec_full['R@10']:.4f}")
print(f"Recall@50: {rec_full['R@50']:.4f}")
print(f"{'='*70}\n")

print("🚀 Done!")


🚀 Using device: cuda
📂 Loading data...
✓ Train captions: (125000, 1024)
✓ Train images:  (25000, 1536)
✓ Test captions:  (1500, 1024)
🔧 Preprocessing embeddings...
✓ Expanded images: torch.Size([125000, 1536])
📏 Creating train/val split (by image id)...
✓ Train pairs: 112500, Val pairs: 12500
✓ Val images: 2500
🏗️ Building model...
📊 Model parameters: 36,221,952
✓ DataLoader ready: 112500 samples

🚀 STARTING TRAINING: simple_baseline_clip



/tmp/ipykernel_48/1058275450.py:263: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if device.type == "cuda" else None


Epoch 01/50:   0%|          | 0/220 [00:00<?, ?it/s]

/tmp/ipykernel_48/1058275450.py:298: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Validation:   0%|          | 0/5 [00:00<?, ?it/s]

...

Epoch 50: Loss=0.1278 | LR=3.0e-07 | R@1=0.3826 R@5=0.6790 R@10=0.7754

🎉 Training complete! Best R@10 (subset): 0.7986

📥 Loading best model from simple_baseline_clip_best.pth...
🔮 Generating test predictions...


Test inference:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Submission saved: submission_simple_baseline_clip.csv

📊 Computing full validation metrics on val split...


Validation:   0%|          | 0/13 [00:00<?, ?it/s]


🎯 FINAL VALIDATION RESULTS (VAL CAPTIONS → VAL IMAGES)
Recall@1:  0.3670
Recall@5:  0.6733
Recall@10: 0.7864
Recall@50: 0.9294

🚀 Done!


In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.cluster import MiniBatchKMeans
from torch.cuda.amp import GradScaler, autocast

# =============================================================================
# ULTIMATE TEXT-IMAGE ALIGNMENT MODEL
# SigLIP-style loss + CUSA-style uni-modal alignment + Relative Reps + Stitching + Light Prior
# =============================================================================

# -----------------------------
# 0. DEVICE
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu_device = torch.device("cpu")
print(f"🚀 Using device: {device}")

# -----------------------------
# 1. DATA LOADING
# -----------------------------
print("📂 Loading data...")

TRAIN_PATH = "/kaggle/input/aml-competition/train/train/train.npz"
TEST_PATH  = "/kaggle/input/aml-competition/test/test/test.clean.npz"

train_data = np.load(TRAIN_PATH)
test_data  = np.load(TEST_PATH)

tx_train = train_data["captions/embeddings"]   # (N_pairs, 1024)
im_train = train_data["images/embeddings"]     # (N_images, 1536)
tx_test  = test_data["captions/embeddings"]    # (N_test, 1024)

print(f"✓ Train captions: {tx_train.shape}")
print(f"✓ Train images:  {im_train.shape}")
print(f"✓ Test captions:  {tx_test.shape}")

# -----------------------------
# 2. PREPROCESSING & NORMALIZATION
# -----------------------------
print("🔧 Preprocessing embeddings...")

tx_train_t        = torch.as_tensor(tx_train, dtype=torch.float32, device=cpu_device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=cpu_device)
tx_test_t         = torch.as_tensor(tx_test,  dtype=torch.float32, device=cpu_device)

# Global mean-centering
tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)

tx_train_t        = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t         = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

# Expand images to match captions (5 captions per image)
repeat_factor = tx_train_t.size(0) // im_train_t_unique.size(0)
assert repeat_factor * im_train_t_unique.size(0) == tx_train_t.size(0), "Non-integer repeat factor"

im_train_exp = im_train_t_unique.repeat_interleave(repeat_factor, dim=0)

print(f"✓ Expanded images: {im_train_exp.shape}")
print("✓ Embeddings centered and L2-normalized")

# -----------------------------
# 3. VALIDATION SPLIT (BY IMAGE ID)
# -----------------------------
print("📏 Creating train/val split (by image id)...")

N_pairs  = tx_train_t.size(0)
N_images = im_train_t_unique.size(0)
val_frac = 0.1
val_img_n = max(1, int(val_frac * N_images))

# Image id for each caption-image pair (0..N_images-1)
pair_img_ids = torch.arange(N_pairs, device=cpu_device) // repeat_factor  # (N_pairs,)

# Randomly split images into train / val
perm_imgs     = torch.randperm(N_images, device=cpu_device)
val_img_ids   = perm_imgs[:val_img_n]          # image ids in val
train_img_ids = perm_imgs[val_img_n:]         # image ids in train

# Build boolean mask over pairs based on image ids
is_val_img = torch.zeros(N_images, dtype=torch.bool, device=cpu_device)
is_val_img[val_img_ids] = True
is_val_pair = is_val_img[pair_img_ids]        # (N_pairs,)

val_idx   = torch.nonzero(is_val_pair, as_tuple=False).squeeze(1)
train_idx = torch.nonzero(~is_val_pair, as_tuple=False).squeeze(1)

# Subsets of captions and expanded images
tx_val_t         = tx_train_t[val_idx]
tx_train_t_sub   = tx_train_t[train_idx]
im_train_exp_sub = im_train_exp[train_idx]

# Image indices for training pairs (GLOBAL ids 0..N_images-1)
img_indices_train = pair_img_ids[train_idx]

# ----- Build validation gallery: only VAL images -----
val_img_ids_sorted, _ = torch.sort(val_img_ids)
im_val = im_train_t_unique[val_img_ids_sorted]   # (N_val_images, 1536)

# Map global image ids -> local indices in [0, N_val_images)
map_global_to_local = -torch.ones(N_images, dtype=torch.long, device=cpu_device)
map_global_to_local[val_img_ids_sorted] = torch.arange(val_img_ids_sorted.size(0), device=cpu_device)

# For each val pair: global -> local image index
val_pair_img_global = pair_img_ids[val_idx]               # global id for each val pair
val_img_local       = map_global_to_local[val_pair_img_global]  # local indices in gallery

# Subset for faster per-epoch validation
subset_size        = min(5000, tx_val_t.size(0))
val_query_subset   = tx_val_t[:subset_size]
val_indices_subset = val_img_local[:subset_size]

print(f"✓ Train pairs: {tx_train_t_sub.size(0)}, Val pairs: {tx_val_t.size(0)}")
print(f"✓ Val images: {im_val.size(0)}")

# -----------------------------
# 4. MODEL ARCHITECTURE
# -----------------------------
class ResidualMLP(nn.Module):
    """Residual MLP block with LayerNorm and GELU."""
    def __init__(self, dim, hidden_dim, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.LayerNorm(dim),
            nn.Dropout(dropout),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x + self.net(x)


class UltimateAlignmentModel(nn.Module):
    """
    Dual projection heads (text & image) + relative reps via anchors +
    shared stitching layer + light Gaussian prior on text.

    Input:
      - text_x: (B, text_dim)
      - img_x:  (B, img_dim)

    Output:
      - training:  (z_t, z_i, mu, logvar)
      - eval:      (z_t, z_i)
    """
    def __init__(
        self,
        text_dim: int = 1024,
        img_dim: int = 1536,
        proj_dim: int = 1536,
        hidden_dim: int = 6144,
        n_anchors: int = 256,
        dropout: float = 0.3,
    ):
        super().__init__()
        self.proj_dim = proj_dim
        self.n_anchors = n_anchors

        # === TEXT PATHWAY ===
        self.text_expand = nn.Linear(text_dim, proj_dim)
        self.text_mlp1   = ResidualMLP(proj_dim, hidden_dim, dropout)
        self.text_mlp2   = ResidualMLP(proj_dim, hidden_dim, dropout)
        self.text_norm   = nn.LayerNorm(proj_dim)

        # Learnable text anchors
        self.text_anchors = nn.Parameter(torch.randn(n_anchors, proj_dim) * 0.02)

        # Light prior (VAE-style)
        self.text_mu     = nn.Linear(proj_dim, proj_dim)
        self.text_logvar = nn.Parameter(torch.zeros(proj_dim))

        # === IMAGE PATHWAY ===
        self.img_expand = nn.Linear(img_dim, proj_dim)
        self.img_mlp1   = ResidualMLP(proj_dim, hidden_dim, dropout)
        self.img_mlp2   = ResidualMLP(proj_dim, hidden_dim, dropout)
        self.img_norm   = nn.LayerNorm(proj_dim)

        # Learnable image anchors
        self.img_anchors = nn.Parameter(torch.randn(n_anchors, proj_dim) * 0.02)

        # === SHARED STITCHING LAYER ===
        concat_dim = proj_dim + n_anchors
        self.stitch = nn.Linear(concat_dim, concat_dim)

        # === FINAL PROJECTION ===
        self.final_norm = nn.LayerNorm(concat_dim)
        self.final_proj = nn.Linear(concat_dim, proj_dim)

    def _relative_representation(self, x: torch.Tensor, anchors: torch.Tensor) -> torch.Tensor:
        x_norm       = F.normalize(x, p=2, dim=1)
        anchors_norm = F.normalize(anchors, p=2, dim=1)
        return x_norm @ anchors_norm.T  # (B, n_anchors)

    def encode_text(self, x: torch.Tensor, return_mu_logvar: bool = False):
        # x: (B, text_dim)
        h = self.text_expand(x)
        h = self.text_mlp1(h)
        h = self.text_mlp2(h)
        h = self.text_norm(h)

        rel = self._relative_representation(h, self.text_anchors)  # (B, n_anchors)
        z   = torch.cat([h, rel], dim=1)                           # (B, proj_dim + n_anchors)

        z = self.stitch(z)
        z = self.final_norm(z)
        z = self.final_proj(z)
        z = F.normalize(z, p=2, dim=1)

        if return_mu_logvar and self.training:
            mu     = self.text_mu(h)
            logvar = self.text_logvar.unsqueeze(0).expand(x.size(0), -1)
            return z, mu, logvar
        return z

    def encode_image(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, img_dim)
        h = self.img_expand(x)
        h = self.img_mlp1(h)
        h = self.img_mlp2(h)
        h = self.img_norm(h)

        rel = self._relative_representation(h, self.img_anchors)  # (B, n_anchors)
        z   = torch.cat([h, rel], dim=1)

        z = self.stitch(z)
        z = self.final_norm(z)
        z = self.final_proj(z)
        z = F.normalize(z, p=2, dim=1)
        return z

    def forward(self, text_x: torch.Tensor, img_x: torch.Tensor):
        if self.training:
            z_t, mu, logvar = self.encode_text(text_x, return_mu_logvar=True)
            z_i = self.encode_image(img_x)
            return z_t, z_i, mu, logvar
        else:
            z_t = self.encode_text(text_x)
            z_i = self.encode_image(img_x)
            return z_t, z_i

# -----------------------------
# 5. LOSS FUNCTIONS
# -----------------------------
class SigLIPLoss(nn.Module):
    """Pairwise logistic loss handling multi-positives via labels."""
    def __init__(self, temperature: float = 0.07):
        super().__init__()
        self.temperature = temperature

    def forward(self, z_t: torch.Tensor, z_i: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        z_t:    (B, D)
        z_i:    (B, D)
        labels: (B,) image index per caption (same for multi-captions)
        """
        logits  = (z_t @ z_i.T) / self.temperature  # (B, B)
        targets = (labels.unsqueeze(1) == labels.unsqueeze(0)).float()  # (B, B)

        # Pairwise logistic loss
        loss_matrix = -targets * F.logsigmoid(logits) - (1 - targets) * F.logsigmoid(-logits)
        return loss_matrix.mean()


class CUSALoss(nn.Module):
    """Uni-modal structure alignment via KL(student || teacher)."""
    def __init__(self, temperature: float = 0.1):
        super().__init__()
        self.temperature = temperature

    def forward(self, student_sim: torch.Tensor, teacher_sim: torch.Tensor) -> torch.Tensor:
        # student_sim, teacher_sim: (B_s, B_s) cosine similarity matrices
        student_logp = F.log_softmax(student_sim / self.temperature, dim=1)
        teacher_p    = F.softmax(teacher_sim / self.temperature, dim=1)
        loss = F.kl_div(student_logp, teacher_p, reduction='batchmean')
        return loss


def compute_kl_loss(mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
    """Standard KL divergence to N(0, I), averaged per example."""
    kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return kl / mu.size(0)

# -----------------------------
# 6. CSLS-BASED VALIDATION (RECALL@K)
# -----------------------------
@torch.no_grad()
def recall_at_k(
    model: nn.Module,
    tx_queries: torch.Tensor,        # [Nq, text_dim], on CPU
    im_database: torch.Tensor,       # [Nd, img_dim], on CPU
    query_img_indices: torch.Tensor, # [Nq], local indices in [0, Nd)
    chunk_size: int = 1024,
    ks=(1, 5, 10, 50),
    k_csls: int = 10,
):
    model.eval()
    dev = next(model.parameters()).device
    all_correct_in_top_k = {k: [] for k in ks}

    # Encode queries
    Q_embed_list = []
    for i in range(0, tx_queries.size(0), chunk_size):
        chunk = tx_queries[i:i + chunk_size].to(dev)
        Q_embed_list.append(model.encode_text(chunk))
    Q_embed = torch.cat(Q_embed_list, dim=0)  # [Nq, D]

    # Encode database
    D_embed_list = []
    for i in range(0, im_database.size(0), chunk_size):
        chunk = im_database[i:i + chunk_size].to(dev)
        D_embed_list.append(model.encode_image(chunk))
    D_embed = torch.cat(D_embed_list, dim=0)  # [Nd, D]

    # CSLS pass 1: mean sim to top-k queries per database vector
    mean_knn_d_list = []
    for i in tqdm(range(0, D_embed.size(0), 512), desc="CSLS (1/2)", leave=False):
        sim_chunk = D_embed[i:i + 512] @ Q_embed.T  # [b_d, Nq]
        knn_vals  = torch.topk(sim_chunk, k=k_csls, dim=1).values
        mean_knn_d_list.append(knn_vals.mean(1))
    mean_knn_d = torch.cat(mean_knn_d_list, dim=0).unsqueeze(0)  # [1, Nd]

    gt = query_img_indices.to(dev)  # [Nq]

    # CSLS pass 2: per query
    for i in tqdm(range(0, Q_embed.size(0), chunk_size), desc="CSLS (2/2)", leave=False):
        q_chunk  = Q_embed[i:i + chunk_size]     # [b_q, D]
        gt_chunk = gt[i:i + chunk_size]          # [b_q]

        sim_chunk = q_chunk @ D_embed.T          # [b_q, Nd]

        knn_q    = torch.topk(sim_chunk, k=k_csls, dim=1).values
        mean_knn_q = knn_q.mean(1, keepdim=True)   # [b_q, 1]

        csls_sim = 2 * sim_chunk - mean_knn_q - mean_knn_d  # broadcast

        top_indices = torch.argsort(csls_sim, dim=1, descending=True)

        for k in ks:
            top_k_preds = top_indices[:, :k]
            correct     = (top_k_preds == gt_chunk.unsqueeze(1)).any(dim=1)
            all_correct_in_top_k[k].append(correct)

    recalls = {}
    for k in ks:
        recalls[f"R@{k}"] = torch.cat(all_correct_in_top_k[k]).float().mean().item()
    return recalls

# -----------------------------
# 7. HYPERPARAMS & MODEL INIT
# -----------------------------
EPOCHS        = 50
BATCH_SIZE    = 512
LR            = 3e-4
WEIGHT_DECAY  = 1e-4
DROPOUT       = 0.3
N_ANCHORS     = 256

WEIGHT_SIGLIP   = 1.0
WEIGHT_USA_TEXT = 0.1
WEIGHT_USA_IMG  = 0.1
WEIGHT_KL       = 1e-7

RUN_ID    = "ultimate_v1_image_split"
SAVE_PATH = f"{RUN_ID}_best.pth"

print("🏗️ Building model...")
model = UltimateAlignmentModel(
    text_dim=tx_train_t.size(1),
    img_dim=im_train_t_unique.size(1),
    proj_dim=im_train_t_unique.size(1),
    hidden_dim=6144,
    n_anchors=N_ANCHORS,
    dropout=DROPOUT,
).to(device)

total_params     = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"📊 Model parameters: {trainable_params:,} trainable / {total_params:,} total")

# -----------------------------
# 8. K-MEANS ANCHOR INITIALIZATION
# -----------------------------
print("🎯 Initializing anchors with k-means on projected space...")

with torch.no_grad():
    sample_size = min(10000, tx_train_t_sub.size(0))
    sample_idx  = torch.randperm(tx_train_t_sub.size(0))[:sample_size]

    # Text projected
    tx_sample      = tx_train_t_sub[sample_idx]
    tx_proj_list   = []
    for i in range(0, tx_sample.size(0), 1024):
        chunk = tx_sample[i:i + 1024].to(device)
        h = model.text_expand(chunk)
        h = model.text_mlp1(h)
        h = model.text_mlp2(h)
        h = model.text_norm(h)
        tx_proj_list.append(h.cpu())
    tx_projected = torch.cat(tx_proj_list, dim=0)

    # Image projected
    img_sample_idx = img_indices_train[sample_idx]  # global image ids
    im_sample      = im_train_t_unique[img_sample_idx]  # CPU
    im_proj_list   = []
    for i in range(0, im_sample.size(0), 1024):
        chunk = im_sample[i:i + 1024].to(device)
        h = model.img_expand(chunk)
        h = model.img_mlp1(h)
        h = model.img_mlp2(h)
        h = model.img_norm(h)
        im_proj_list.append(h.cpu())
    im_projected = torch.cat(im_proj_list, dim=0)

    print("  Running k-means on text projections...")
    kmeans_text = MiniBatchKMeans(n_clusters=N_ANCHORS, batch_size=1024,
                                  random_state=42, n_init=3)
    text_anchors_init = kmeans_text.fit(tx_projected.numpy()).cluster_centers_
    text_anchors_init = F.normalize(
        torch.from_numpy(text_anchors_init).float(), p=2, dim=1
    )

    print("  Running k-means on image projections...")
    kmeans_img = MiniBatchKMeans(n_clusters=N_ANCHORS, batch_size=1024,
                                 random_state=42, n_init=3)
    img_anchors_init = kmeans_img.fit(im_projected.numpy()).cluster_centers_
    img_anchors_init = F.normalize(
        torch.from_numpy(img_anchors_init).float(), p=2, dim=1
    )

    model.text_anchors.data = text_anchors_init.to(device)
    model.img_anchors.data  = img_anchors_init.to(device)

print("✓ Anchors initialized.")

# -----------------------------
# 9. TRAINING OBJECTS (OPTIM, SCHED, SCALER, DATA)
# -----------------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

scaler = GradScaler() if device.type == "cuda" else None

siglip_loss_fn = SigLIPLoss(temperature=0.07)
cusa_loss_fn   = CUSALoss(temperature=0.1)

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader  = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
)

print(f"✓ DataLoader ready: {len(train_dataset)} training samples")

# -----------------------------
# 10. TRAINING LOOP
# -----------------------------
print(f"\n{'='*70}")
print(f"🚀 STARTING TRAINING: {RUN_ID}")
print(f"{'='*70}\n")

best_val_r10 = 0.0

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_siglip = total_usa_text = total_usa_img = total_kl = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch:02d}/{EPOCHS}", leave=False)

    for batch_idx, (tx_batch, im_batch, img_labels) in enumerate(pbar):
        tx_batch   = tx_batch.to(device)
        im_batch   = im_batch.to(device)
        img_labels = img_labels.to(device)

        optimizer.zero_grad()

        if scaler is not None:
            with autocast():
                z_t, z_i, mu, logvar = model(tx_batch, im_batch)
                loss_siglip = siglip_loss_fn(z_t, z_i, img_labels)

                # CUSA: every 4th batch, on a subset
                loss_usa_text = torch.tensor(0.0, device=device)
                loss_usa_img  = torch.tensor(0.0, device=device)

                if batch_idx % 4 == 0 and tx_batch.size(0) >= 256:
                    n_sample = min(256, tx_batch.size(0))
                    s_idx    = torch.randperm(tx_batch.size(0), device=device)[:n_sample]

                    # ---- Text CUSA ----
                    tx_teacher      = tx_batch[s_idx]            # teacher in original text space
                    z_t_student     = model.encode_text(tx_teacher)  # student in aligned space
                    student_sim_tt  = z_t_student @ z_t_student.T
                    teacher_sim_tt  = tx_teacher @ tx_teacher.T
                    loss_usa_text   = cusa_loss_fn(student_sim_tt, teacher_sim_tt)

                    # ---- Image CUSA ----
                    img_ids_sample   = img_labels[s_idx]  # global ids
                    im_teacher       = im_train_t_unique[img_ids_sample.cpu()].to(device)
                    z_i_student      = model.encode_image(im_teacher)
                    student_sim_ii   = z_i_student @ z_i_student.T
                    teacher_sim_ii   = im_teacher @ im_teacher.T
                    loss_usa_img     = cusa_loss_fn(student_sim_ii, teacher_sim_ii)

                loss_kl = compute_kl_loss(mu, logvar)

                loss = (
                    WEIGHT_SIGLIP   * loss_siglip +
                    WEIGHT_USA_TEXT * loss_usa_text +
                    WEIGHT_USA_IMG  * loss_usa_img +
                    WEIGHT_KL       * loss_kl
                )

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
        else:
            # CPU / no-AMP path
            z_t, z_i, mu, logvar = model(tx_batch, im_batch)
            loss_siglip = siglip_loss_fn(z_t, z_i, img_labels)

            loss_usa_text = torch.tensor(0.0, device=device)
            loss_usa_img  = torch.tensor(0.0, device=device)

            if batch_idx % 4 == 0 and tx_batch.size(0) >= 256:
                n_sample = min(256, tx_batch.size(0))
                s_idx    = torch.randperm(tx_batch.size(0), device=device)[:n_sample]

                tx_teacher     = tx_batch[s_idx]
                z_t_student    = model.encode_text(tx_teacher)
                student_sim_tt = z_t_student @ z_t_student.T
                teacher_sim_tt = tx_teacher @ tx_teacher.T
                loss_usa_text  = cusa_loss_fn(student_sim_tt, teacher_sim_tt)

                img_ids_sample = img_labels[s_idx]
                im_teacher     = im_train_t_unique[img_ids_sample.cpu()].to(device)
                z_i_student    = model.encode_image(im_teacher)
                student_sim_ii = z_i_student @ z_i_student.T
                teacher_sim_ii = im_teacher @ im_teacher.T
                loss_usa_img   = cusa_loss_fn(student_sim_ii, teacher_sim_ii)

            loss_kl = compute_kl_loss(mu, logvar)

            loss = (
                WEIGHT_SIGLIP   * loss_siglip +
                WEIGHT_USA_TEXT * loss_usa_text +
                WEIGHT_USA_IMG  * loss_usa_img +
                WEIGHT_KL       * loss_kl
            )

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        total_siglip   += loss_siglip.item()
        total_usa_text += loss_usa_text.item()
        total_usa_img  += loss_usa_img.item()
        total_kl       += loss_kl.item()

        pbar.set_postfix({
            'SigLIP': f'{loss_siglip.item():.3f}',
            'USA_T':  f'{loss_usa_text.item():.3f}',
            'USA_I':  f'{loss_usa_img.item():.3f}',
            'KL':     f'{loss_kl.item():.1e}',
        })

    current_lr = scheduler.get_last_lr()[0]
    scheduler.step()

    avg_siglip   = total_siglip   / len(train_loader)
    avg_usa_text = total_usa_text / len(train_loader)
    avg_usa_img  = total_usa_img  / len(train_loader)
    avg_kl       = total_kl       / len(train_loader)

    # Validation on subset (val captions → val images only)
    rec = recall_at_k(
        model,
        val_query_subset,
        im_val,
        val_indices_subset,
        chunk_size=1024,
    )
    current_r10 = rec['R@10']

    save_marker = ""
    if current_r10 > best_val_r10:
        best_val_r10 = current_r10
        torch.save(model.state_dict(), SAVE_PATH)
        save_marker = " ✨ BEST"

    print(
        f"Epoch {epoch:02d}: "
        f"SigLIP={avg_siglip:.4f} "
        f"USA_T={avg_usa_text:.4f} "
        f"USA_I={avg_usa_img:.4f} "
        f"KL={avg_kl:.1e} "
        f"| LR={current_lr:.1e} "
        f"| R@1={rec['R@1']:.4f} R@5={rec['R@5']:.4f} R@10={current_r10:.4f}{save_marker}"
    )

print(f"\n{'='*70}")
print(f"🎉 Training complete! Best R@10 (subset): {best_val_r10:.4f}")
print(f"{'='*70}\n")

# -----------------------------
# 11. FINAL INFERENCE & SUBMISSION
# -----------------------------
print(f"📥 Loading best model from {SAVE_PATH}...")
model.load_state_dict(torch.load(SAVE_PATH, map_location=device))
model.to(device)
model.eval()

print("🔮 Generating test predictions...")
with torch.no_grad():
    preds_list = []
    for i in tqdm(range(0, tx_test_t.size(0), 1024), desc="Test inference"):
        chunk = tx_test_t[i:i + 1024].to(device)
        preds_list.append(model.encode_text(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)

submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds],
})

submission_filename = f"submission_{RUN_ID}.csv"
submission.to_csv(submission_filename, index=False)
print(f"✅ Submission saved: {submission_filename}")

# -----------------------------
# 12. FULL VALIDATION METRICS (VAL CAPTIONS → VAL IMAGES)
# -----------------------------
print("\n📊 Computing full validation metrics on val split...")
rec_full = recall_at_k(
    model,
    tx_val_t,
    im_val,
    val_img_local,
    chunk_size=1024,
)

print(f"\n{'='*70}")
print("🎯 FINAL VALIDATION RESULTS (VAL CAPTIONS → VAL IMAGES)")
print(f"{'='*70}")
print(f"Recall@1:  {rec_full['R@1']:.4f}")
print(f"Recall@5:  {rec_full['R@5']:.4f}")
print(f"Recall@10: {rec_full['R@10']:.4f}")
print(f"Recall@50: {rec_full['R@50']:.4f}")
print(f"{'='*70}\n")

print("🚀 Done! Ultimate alignment model training + inference complete.")


🚀 Using device: cuda
📂 Loading data...
✓ Train captions: (125000, 1024)
✓ Train images:  (25000, 1536)
✓ Test captions:  (1500, 1024)
🔧 Preprocessing embeddings...
✓ Expanded images: torch.Size([125000, 1536])
✓ Embeddings centered and L2-normalized
📏 Creating train/val split (by image id)...
✓ Train pairs: 112500, Val pairs: 12500
✓ Val images: 2500
🏗️ Building model...
📊 Model parameters: 88,650,496 trainable / 88,650,496 total
🎯 Initializing anchors with k-means on projected space...
  Running k-means on text projections...
  Running k-means on image projections...
✓ Anchors initialized.
✓ DataLoader ready: 112500 training samples

🚀 STARTING TRAINING: ultimate_v1_image_split



/tmp/ipykernel_48/4038462582.py:451: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if device.type == "cuda" else None


Epoch 01/50:   0%|          | 0/220 [00:00<?, ?it/s]

/tmp/ipykernel_48/4038462582.py:489: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


CSLS (1/2):   0%|          | 0/5 [00:00<?, ?it/s]

...

Test inference:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Submission saved: submission_ultimate_v1_image_split.csv

📊 Computing full validation metrics on val split...


CSLS (1/2):   0%|          | 0/5 [00:00<?, ?it/s]

CSLS (2/2):   0%|          | 0/13 [00:00<?, ?it/s]


🎯 FINAL VALIDATION RESULTS (VAL CAPTIONS → VAL IMAGES)
Recall@1:  0.0412
Recall@5:  0.1375
Recall@10: 0.2182
Recall@50: 0.5122

🚀 Done! Ultimate alignment model training + inference complete.


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.cluster import MiniBatchKMeans
from torch.cuda.amp import GradScaler, autocast

# =============================================================================
# ULTIMATE TEXT → IMAGE-SPACE ALIGNMENT MODEL
# - Trains ONLY the text tower to match FIXED DINO image embeddings
# - SigLIP-style loss in RAW DINO space
# - CUSA-style uni-modal alignment (text + optional image)
# - Relative representations via anchors
# - Stitching + light prior
# =============================================================================

# -----------------------------
# 0. DEVICE
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu_device = torch.device("cpu")
print(f"🚀 Using device: {device}")

# -----------------------------
# 1. DATA LOADING
# -----------------------------
print("📂 Loading data...")

TRAIN_PATH = "/kaggle/input/aml-competition/train/train/train.npz"
TEST_PATH  = "/kaggle/input/aml-competition/test/test/test.clean.npz"

train_data = np.load(TRAIN_PATH)
test_data  = np.load(TEST_PATH)

tx_train = train_data["captions/embeddings"]   # (N_pairs, 1024)
im_train = train_data["images/embeddings"]     # (N_images, 1536)
tx_test  = test_data["captions/embeddings"]    # (N_test, 1024)

print(f"✓ Train captions: {tx_train.shape}")
print(f"✓ Train images:  {im_train.shape}")
print(f"✓ Test captions:  {tx_test.shape}")

# -----------------------------
# 2. PREPROCESSING & NORMALIZATION
# -----------------------------
print("🔧 Preprocessing embeddings...")

tx_train_t        = torch.as_tensor(tx_train, dtype=torch.float32, device=cpu_device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=cpu_device)
tx_test_t         = torch.as_tensor(tx_test,  dtype=torch.float32, device=cpu_device)

# Global mean-centering
tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)

tx_train_t        = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t         = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

# Expand images to match captions (5 captions per image)
repeat_factor = tx_train_t.size(0) // im_train_t_unique.size(0)
assert repeat_factor * im_train_t_unique.size(0) == tx_train_t.size(0), "Non-integer repeat factor"

im_train_exp = im_train_t_unique.repeat_interleave(repeat_factor, dim=0)

print(f"✓ Expanded images: {im_train_exp.shape}")
print("✓ Embeddings centered and L2-normalized")

# -----------------------------
# 3. VALIDATION SPLIT (BY IMAGE ID)
# -----------------------------
print("📏 Creating train/val split (by image id)...")

N_pairs  = tx_train_t.size(0)
N_images = im_train_t_unique.size(0)
val_frac = 0.1
val_img_n = max(1, int(val_frac * N_images))

# Image id for each caption-image pair (0..N_images-1)
pair_img_ids = torch.arange(N_pairs, device=cpu_device) // repeat_factor  # (N_pairs,)

# Randomly split images into train / val
perm_imgs     = torch.randperm(N_images, device=cpu_device)
val_img_ids   = perm_imgs[:val_img_n]         # image ids in val
train_img_ids = perm_imgs[val_img_n:]         # image ids in train

# Build boolean mask over pairs based on image ids
is_val_img = torch.zeros(N_images, dtype=torch.bool, device=cpu_device)
is_val_img[val_img_ids] = True
is_val_pair = is_val_img[pair_img_ids]        # (N_pairs,)

val_idx   = torch.nonzero(is_val_pair, as_tuple=False).squeeze(1)
train_idx = torch.nonzero(~is_val_pair, as_tuple=False).squeeze(1)

# Subsets of captions and expanded images
tx_val_t         = tx_train_t[val_idx]
tx_train_t_sub   = tx_train_t[train_idx]
im_train_exp_sub = im_train_exp[train_idx]

# Image indices for training pairs (GLOBAL ids 0..N_images-1)
img_indices_train = pair_img_ids[train_idx]

# ----- Build validation gallery: only VAL images -----
val_img_ids_sorted, _ = torch.sort(val_img_ids)
im_val = im_train_t_unique[val_img_ids_sorted]   # (N_val_images, 1536)

# Map global image ids -> local indices in [0, N_val_images)
map_global_to_local = -torch.ones(N_images, dtype=torch.long, device=cpu_device)
map_global_to_local[val_img_ids_sorted] = torch.arange(val_img_ids_sorted.size(0), device=cpu_device)

# For each val pair: global -> local image index
val_pair_img_global = pair_img_ids[val_idx]               # global id for each val pair
val_img_local       = map_global_to_local[val_pair_img_global]  # local indices in gallery

# Subset for faster per-epoch validation
subset_size        = min(5000, tx_val_t.size(0))
val_query_subset   = tx_val_t[:subset_size]
val_indices_subset = val_img_local[:subset_size]

print(f"✓ Train pairs: {tx_train_t_sub.size(0)}, Val pairs: {tx_val_t.size(0)}")
print(f"✓ Val images: {im_val.size(0)}")

# -----------------------------
# 4. MODEL ARCHITECTURE
# -----------------------------
class ResidualMLP(nn.Module):
    """Residual MLP block with LayerNorm and GELU."""
    def __init__(self, dim, hidden_dim, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.LayerNorm(dim),
            nn.Dropout(dropout),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x + self.net(x)


class UltimateAlignmentModel(nn.Module):
    """
    TEXT-ONLY alignment head that maps frozen text embeddings -> DINO image space.

    - Text & image projectors exist, but only the TEXT output is used
      in the main SigLIP loss.
    - Image pathway is used only for anchor init + optional CUSA regularization.

    Inputs:
      - encode_text: (B, text_dim) -> (B, proj_dim)
      - encode_image: (B, img_dim) -> (B, proj_dim) [auxiliary]
      - forward(text_x): (B, text_dim) -> (z_t, mu, logvar)
    """
    def __init__(
        self,
        text_dim: int = 1024,
        img_dim: int = 1536,
        proj_dim: int = 1536,
        hidden_dim: int = 6144,
        n_anchors: int = 256,
        dropout: float = 0.3,
    ):
        super().__init__()
        self.proj_dim = proj_dim
        self.n_anchors = n_anchors

        # === TEXT PATHWAY ===
        self.text_expand = nn.Linear(text_dim, proj_dim)
        self.text_mlp1   = ResidualMLP(proj_dim, hidden_dim, dropout)
        self.text_mlp2   = ResidualMLP(proj_dim, hidden_dim, dropout)
        self.text_norm   = nn.LayerNorm(proj_dim)

        # Learnable text anchors
        self.text_anchors = nn.Parameter(torch.randn(n_anchors, proj_dim) * 0.02)

        # Light prior (VAE-style)
        self.text_mu     = nn.Linear(proj_dim, proj_dim)
        self.text_logvar = nn.Parameter(torch.zeros(proj_dim))

        # === IMAGE PATHWAY (AUXILIARY ONLY) ===
        self.img_expand = nn.Linear(img_dim, proj_dim)
        self.img_mlp1   = ResidualMLP(proj_dim, hidden_dim, dropout)
        self.img_mlp2   = ResidualMLP(proj_dim, hidden_dim, dropout)
        self.img_norm   = nn.LayerNorm(proj_dim)

        # Learnable image anchors
        self.img_anchors = nn.Parameter(torch.randn(n_anchors, proj_dim) * 0.02)

        # === SHARED STITCHING LAYER ===
        concat_dim = proj_dim + n_anchors
        self.stitch = nn.Linear(concat_dim, concat_dim)

        # === FINAL PROJECTION (shared for text / image heads) ===
        self.final_norm = nn.LayerNorm(concat_dim)
        self.final_proj = nn.Linear(concat_dim, proj_dim)

    def _relative_representation(self, x: torch.Tensor, anchors: torch.Tensor) -> torch.Tensor:
        x_norm       = F.normalize(x, p=2, dim=1)
        anchors_norm = F.normalize(anchors, p=2, dim=1)
        return x_norm @ anchors_norm.T  # (B, n_anchors)

    def encode_text(self, x: torch.Tensor, return_mu_logvar: bool = False):
        # x: (B, text_dim)
        h = self.text_expand(x)
        h = self.text_mlp1(h)
        h = self.text_mlp2(h)
        h = self.text_norm(h)

        rel = self._relative_representation(h, self.text_anchors)  # (B, n_anchors)
        z   = torch.cat([h, rel], dim=1)                           # (B, proj_dim + n_anchors)

        z = self.stitch(z)
        z = self.final_norm(z)
        z = self.final_proj(z)
        z = F.normalize(z, p=2, dim=1)

        if return_mu_logvar and self.training:
            mu     = self.text_mu(h)
            logvar = self.text_logvar.unsqueeze(0).expand(x.size(0), -1)
            return z, mu, logvar
        return z

    def encode_image(self, x: torch.Tensor) -> torch.Tensor:
        """
        Auxiliary image encoder. NOT used for main SigLIP objective,
        only for CUSA regularization and anchor initialization.
        """
        h = self.img_expand(x)
        h = self.img_mlp1(h)
        h = self.img_mlp2(h)
        h = self.img_norm(h)

        rel = self._relative_representation(h, self.img_anchors)  # (B, n_anchors)
        z   = torch.cat([h, rel], dim=1)

        z = self.stitch(z)
        z = self.final_norm(z)
        z = self.final_proj(z)
        z = F.normalize(z, p=2, dim=1)
        return z

    def forward(self, text_x: torch.Tensor):
        """
        Forward pass used during training for the main objective.
        Only processes TEXT and returns:
          - z_t:    aligned text embedding in image space
          - mu,logvar: for KL prior on text
        """
        z_t, mu, logvar = self.encode_text(text_x, return_mu_logvar=True)
        return z_t, mu, logvar

# -----------------------------
# 5. LOSS FUNCTIONS
# -----------------------------
class SigLIPLoss(nn.Module):
    """Pairwise logistic loss handling multi-positives via labels."""
    def __init__(self, temperature: float = 0.07):
        super().__init__()
        self.temperature = temperature

    def forward(self, z_t: torch.Tensor, z_i: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        z_t:    (B, D) text embeddings (learned)
        z_i:    (B, D) image embeddings (here: RAW DINO, normalized)
        labels: (B,)  image index per caption (same for multi-captions)
        """
        logits  = (z_t @ z_i.T) / self.temperature  # (B, B)
        targets = (labels.unsqueeze(1) == labels.unsqueeze(0)).float()  # (B, B)

        # Pairwise logistic loss
        loss_matrix = -targets * F.logsigmoid(logits) - (1 - targets) * F.logsigmoid(-logits)
        return loss_matrix.mean()


class CUSALoss(nn.Module):
    """Uni-modal structure alignment via KL(student || teacher)."""
    def __init__(self, temperature: float = 0.1):
        super().__init__()
        self.temperature = temperature

    def forward(self, student_sim: torch.Tensor, teacher_sim: torch.Tensor) -> torch.Tensor:
        # student_sim, teacher_sim: (B_s, B_s) cosine similarity matrices
        student_logp = F.log_softmax(student_sim / self.temperature, dim=1)
        teacher_p    = F.softmax(teacher_sim / self.temperature, dim=1)
        loss = F.kl_div(student_logp, teacher_p, reduction='batchmean')
        return loss


def compute_kl_loss(mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
    """Standard KL divergence to N(0, I), averaged per example."""
    kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return kl / mu.size(0)

# -----------------------------
# 6. CSLS-BASED VALIDATION (RECALL@K IN DINO SPACE)
# -----------------------------
@torch.no_grad()
def recall_at_k(
    model: nn.Module,
    tx_queries: torch.Tensor,        # [Nq, text_dim], on CPU
    im_database: torch.Tensor,       # [Nd, img_dim], on CPU (RAW DINO, normalized)
    query_img_indices: torch.Tensor, # [Nq], local indices in [0, Nd)
    chunk_size: int = 1024,
    ks=(1, 5, 10, 50),
    k_csls: int = 10,
):
    """
    IMPORTANT: Database embeddings are RAW (frozen) DINO vectors.
    We only pass queries through the text encoder; images are never
    re-encoded by the model here.
    """
    model.eval()
    dev = next(model.parameters()).device
    all_correct_in_top_k = {k: [] for k in ks}

    # Encode queries
    Q_embed_list = []
    for i in range(0, tx_queries.size(0), chunk_size):
        chunk = tx_queries[i:i + chunk_size].to(dev)
        Q_embed_list.append(model.encode_text(chunk))
    Q_embed = torch.cat(Q_embed_list, dim=0)  # [Nq, D]

    # Database: use RAW DINO embeddings, just L2-normalized
    D_embed_list = []
    for i in range(0, im_database.size(0), chunk_size):
        chunk = im_database[i:i + chunk_size].to(dev)
        D_embed_list.append(F.normalize(chunk, p=2, dim=1))
    D_embed = torch.cat(D_embed_list, dim=0)  # [Nd, D]

    # CSLS pass 1: mean sim to top-k queries per database vector
    mean_knn_d_list = []
    for i in tqdm(range(0, D_embed.size(0), 512), desc="CSLS (1/2)", leave=False):
        sim_chunk = D_embed[i:i + 512] @ Q_embed.T  # [b_d, Nq]
        knn_vals  = torch.topk(sim_chunk, k=k_csls, dim=1).values
        mean_knn_d_list.append(knn_vals.mean(1))
    mean_knn_d = torch.cat(mean_knn_d_list, dim=0).unsqueeze(0)  # [1, Nd]

    gt = query_img_indices.to(dev)  # [Nq]

    # CSLS pass 2: per query
    for i in tqdm(range(0, Q_embed.size(0), chunk_size), desc="CSLS (2/2)", leave=False):
        q_chunk  = Q_embed[i:i + chunk_size]     # [b_q, D]
        gt_chunk = gt[i:i + chunk_size]          # [b_q]

        sim_chunk = q_chunk @ D_embed.T          # [b_q, Nd]

        knn_q      = torch.topk(sim_chunk, k=k_csls, dim=1).values
        mean_knn_q = knn_q.mean(1, keepdim=True)   # [b_q, 1]

        csls_sim = 2 * sim_chunk - mean_knn_q - mean_knn_d  # broadcast

        top_indices = torch.argsort(csls_sim, dim=1, descending=True)

        for k in ks:
            top_k_preds = top_indices[:, :k]
            correct     = (top_k_preds == gt_chunk.unsqueeze(1)).any(dim=1)
            all_correct_in_top_k[k].append(correct)

    recalls = {}
    for k in ks:
        recalls[f"R@k".replace("k", str(k))] = torch.cat(all_correct_in_top_k[k]).float().mean().item()
    return recalls

# -----------------------------
# 7. HYPERPARAMS & MODEL INIT
# -----------------------------
EPOCHS        = 50
BATCH_SIZE    = 512
LR            = 3e-4
WEIGHT_DECAY  = 1e-4
DROPOUT       = 0.3
N_ANCHORS     = 256

WEIGHT_SIGLIP   = 1.0
WEIGHT_USA_TEXT = 0.1
WEIGHT_USA_IMG  = 0.1   # set to 0.0 if you want to drop image-side CUSA
WEIGHT_KL       = 1e-7

RUN_ID    = "ultimate_v2_text_to_dino"
SAVE_PATH = f"{RUN_ID}_best.pth"

print("🏗️ Building model...")
model = UltimateAlignmentModel(
    text_dim=tx_train_t.size(1),
    img_dim=im_train_t_unique.size(1),
    proj_dim=im_train_t_unique.size(1),
    hidden_dim=6144,
    n_anchors=N_ANCHORS,
    dropout=DROPOUT,
).to(device)

total_params     = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"📊 Model parameters: {trainable_params:,} trainable / {total_params:,} total")

# -----------------------------
# 8. K-MEANS ANCHOR INITIALIZATION
# -----------------------------
print("🎯 Initializing anchors with k-means on projected space...")

with torch.no_grad():
    sample_size = min(10000, tx_train_t_sub.size(0))
    sample_idx  = torch.randperm(tx_train_t_sub.size(0))[:sample_size]

    # Text projected
    tx_sample      = tx_train_t_sub[sample_idx]
    tx_proj_list   = []
    for i in range(0, tx_sample.size(0), 1024):
        chunk = tx_sample[i:i + 1024].to(device)
        h = model.text_expand(chunk)
        h = model.text_mlp1(h)
        h = model.text_mlp2(h)
        h = model.text_norm(h)
        tx_proj_list.append(h.cpu())
    tx_projected = torch.cat(tx_proj_list, dim=0)

    # Image projected (auxiliary head)
    img_sample_idx = img_indices_train[sample_idx]  # global image ids
    im_sample      = im_train_t_unique[img_sample_idx]  # CPU
    im_proj_list   = []
    for i in range(0, im_sample.size(0), 1024):
        chunk = im_sample[i:i + 1024].to(device)
        h = model.img_expand(chunk)
        h = model.img_mlp1(h)
        h = model.img_mlp2(h)
        h = model.img_norm(h)
        im_proj_list.append(h.cpu())
    im_projected = torch.cat(im_proj_list, dim=0)

    print("  Running k-means on text projections...")
    kmeans_text = MiniBatchKMeans(n_clusters=N_ANCHORS, batch_size=1024,
                                  random_state=42, n_init=3)
    text_anchors_init = kmeans_text.fit(tx_projected.numpy()).cluster_centers_
    text_anchors_init = F.normalize(
        torch.from_numpy(text_anchors_init).float(), p=2, dim=1
    )

    print("  Running k-means on image projections...")
    kmeans_img = MiniBatchKMeans(n_clusters=N_ANCHORS, batch_size=1024,
                                 random_state=42, n_init=3)
    img_anchors_init = kmeans_img.fit(im_projected.numpy()).cluster_centers_
    img_anchors_init = F.normalize(
        torch.from_numpy(img_anchors_init).float(), p=2, dim=1
    )

    model.text_anchors.data = text_anchors_init.to(device)
    model.img_anchors.data  = img_anchors_init.to(device)

print("✓ Anchors initialized.")

# -----------------------------
# 9. TRAINING OBJECTS (OPTIM, SCHED, SCALER, DATA)
# -----------------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

scaler = GradScaler() if device.type == "cuda" else None

siglip_loss_fn = SigLIPLoss(temperature=0.07)
cusa_loss_fn   = CUSALoss(temperature=0.1)

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader  = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
)

print(f"✓ DataLoader ready: {len(train_dataset)} training samples")

# -----------------------------
# 10. TRAINING LOOP
# -----------------------------
print(f"\n{'='*70}")
print(f"🚀 STARTING TRAINING: {RUN_ID}")
print(f"{'='*70}\n")

best_val_r10 = 0.0

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_siglip = total_usa_text = total_usa_img = total_kl = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch:02d}/{EPOCHS}", leave=False)

    for batch_idx, (tx_batch, im_batch, img_labels) in enumerate(pbar):
        tx_batch   = tx_batch.to(device)
        im_batch   = im_batch.to(device)         # RAW DINO (pre-normalized)
        img_labels = img_labels.to(device)

        optimizer.zero_grad()

        if scaler is not None:
            with autocast():
                # TEXT-ONLY forward
                z_t, mu, logvar = model(tx_batch)

                # IMAGE SIDE: RAW DINO embeddings, L2-normalized
                z_i = F.normalize(im_batch, p=2, dim=1)

                # SigLIP main loss
                loss_siglip = siglip_loss_fn(z_t, z_i, img_labels)

                # CUSA: every 4th batch, on a subset
                loss_usa_text = torch.tensor(0.0, device=device)
                loss_usa_img  = torch.tensor(0.0, device=device)

                if batch_idx % 4 == 0 and tx_batch.size(0) >= 256:
                    n_sample = min(256, tx_batch.size(0))
                    s_idx    = torch.randperm(tx_batch.size(0), device=device)[:n_sample]

                    # ---- Text CUSA ----
                    tx_teacher      = tx_batch[s_idx]               # teacher in original text space
                    z_t_student     = model.encode_text(tx_teacher) # student in aligned space
                    student_sim_tt  = z_t_student @ z_t_student.T
                    teacher_sim_tt  = tx_teacher @ tx_teacher.T
                    loss_usa_text   = cusa_loss_fn(student_sim_tt, teacher_sim_tt)

                    # ---- Image CUSA (auxiliary, optional) ----
                    if WEIGHT_USA_IMG > 0.0:
                        img_ids_sample   = img_labels[s_idx]  # global ids
                        im_teacher       = im_train_t_unique[img_ids_sample.cpu()].to(device)
                        z_i_student      = model.encode_image(im_teacher)
                        student_sim_ii   = z_i_student @ z_i_student.T
                        teacher_sim_ii   = im_teacher @ im_teacher.T
                        loss_usa_img     = cusa_loss_fn(student_sim_ii, teacher_sim_ii)

                loss_kl = compute_kl_loss(mu, logvar)

                loss = (
                    WEIGHT_SIGLIP   * loss_siglip +
                    WEIGHT_USA_TEXT * loss_usa_text +
                    WEIGHT_USA_IMG  * loss_usa_img +
                    WEIGHT_KL       * loss_kl
                )

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
        else:
            # CPU / no-AMP path
            z_t, mu, logvar = model(tx_batch)
            z_i = F.normalize(im_batch, p=2, dim=1)

            loss_siglip = siglip_loss_fn(z_t, z_i, img_labels)

            loss_usa_text = torch.tensor(0.0, device=device)
            loss_usa_img  = torch.tensor(0.0, device=device)

            if batch_idx % 4 == 0 and tx_batch.size(0) >= 256:
                n_sample = min(256, tx_batch.size(0))
                s_idx    = torch.randperm(tx_batch.size(0), device=device)[:n_sample]

                # Text CUSA
                tx_teacher     = tx_batch[s_idx]
                z_t_student    = model.encode_text(tx_teacher)
                student_sim_tt = z_t_student @ z_t_student.T
                teacher_sim_tt = tx_teacher @ tx_teacher.T
                loss_usa_text  = cusa_loss_fn(student_sim_tt, teacher_sim_tt)

                # Image CUSA (aux)
                if WEIGHT_USA_IMG > 0.0:
                    img_ids_sample = img_labels[s_idx]
                    im_teacher     = im_train_t_unique[img_ids_sample.cpu()].to(device)
                    z_i_student    = model.encode_image(im_teacher)
                    student_sim_ii = z_i_student @ z_i_student.T
                    teacher_sim_ii = im_teacher @ im_teacher.T
                    loss_usa_img   = cusa_loss_fn(student_sim_ii, teacher_sim_ii)

            loss_kl = compute_kl_loss(mu, logvar)

            loss = (
                WEIGHT_SIGLIP   * loss_siglip +
                WEIGHT_USA_TEXT * loss_usa_text +
                WEIGHT_USA_IMG  * loss_usa_img +
                WEIGHT_KL       * loss_kl
            )

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        total_siglip   += loss_siglip.item()
        total_usa_text += loss_usa_text.item()
        total_usa_img  += loss_usa_img.item()
        total_kl       += loss_kl.item()

        pbar.set_postfix({
            'SigLIP': f'{loss_siglip.item():.3f}',
            'USA_T':  f'{loss_usa_text.item():.3f}',
            'USA_I':  f'{loss_usa_img.item():.3f}',
            'KL':     f'{loss_kl.item():.1e}',
        })

    current_lr = scheduler.get_last_lr()[0]
    scheduler.step()

    avg_siglip   = total_siglip   / len(train_loader)
    avg_usa_text = total_usa_text / len(train_loader)
    avg_usa_img  = total_usa_img  / len(train_loader)
    avg_kl       = total_kl       / len(train_loader)

    # Validation on subset (val captions → val images only, in RAW DINO space)
    rec = recall_at_k(
        model,
        val_query_subset,
        im_val,
        val_indices_subset,
        chunk_size=1024,
    )
    current_r10 = rec['R@10']

    save_marker = ""
    if current_r10 > best_val_r10:
        best_val_r10 = current_r10
        torch.save(model.state_dict(), SAVE_PATH)
        save_marker = " ✨ BEST"

    print(
        f"Epoch {epoch:02d}: "
        f"SigLIP={avg_siglip:.4f} "
        f"USA_T={avg_usa_text:.4f} "
        f"USA_I={avg_usa_img:.4f} "
        f"KL={avg_kl:.1e} "
        f"| LR={current_lr:.1e} "
        f"| R@1={rec['R@1']:.4f} R@5={rec['R@5']:.4f} R@10={current_r10:.4f}{save_marker}"
    )

print(f"\n{'='*70}")
print(f"🎉 Training complete! Best R@10 (subset, DINO space): {best_val_r10:.4f}")
print(f"{'='*70}\n")

# -----------------------------
# 11. FINAL INFERENCE & SUBMISSION
# -----------------------------
print(f"📥 Loading best model from {SAVE_PATH}...")
model.load_state_dict(torch.load(SAVE_PATH, map_location=device))
model.to(device)
model.eval()

print("🔮 Generating test predictions...")
with torch.no_grad():
    preds_list = []
    for i in tqdm(range(0, tx_test_t.size(0), 1024), desc="Test inference"):
        chunk = tx_test_t[i:i + 1024].to(device)
        preds_list.append(model.encode_text(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)

submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds],
})

submission_filename = f"submission_{RUN_ID}.csv"
submission.to_csv(submission_filename, index=False)
print(f"✅ Submission saved: {submission_filename}")

# -----------------------------
# 12. FULL VALIDATION METRICS (VAL CAPTIONS → VAL IMAGES, DINO SPACE)
# -----------------------------
print("\n📊 Computing full validation metrics on val split...")
rec_full = recall_at_k(
    model,
    tx_val_t,
    im_val,
    val_img_local,
    chunk_size=1024,
)

print(f"\n{'='*70}")
print("🎯 FINAL VALIDATION RESULTS (VAL CAPTIONS → VAL IMAGES, DINO SPACE)")
print(f"{'='*70}")
print(f"Recall@1:  {rec_full['R@1']:.4f}")
print(f"Recall@5:  {rec_full['R@5']:.4f}")
print(f"Recall@10: {rec_full['R@10']:.4f}")
print(f"Recall@50: {rec_full['R@50']:.4f}")
print(f"{'='*70}\n")

print("🚀 Done! Ultimate text→DINO alignment model training + inference complete.")


🚀 Using device: cuda
📂 Loading data...
✓ Train captions: (125000, 1024)
✓ Train images:  (25000, 1536)
✓ Test captions:  (1500, 1024)
🔧 Preprocessing embeddings...
✓ Expanded images: torch.Size([125000, 1536])
✓ Embeddings centered and L2-normalized
📏 Creating train/val split (by image id)...
✓ Train pairs: 112500, Val pairs: 12500
✓ Val images: 2500
🏗️ Building model...
📊 Model parameters: 88,650,496 trainable / 88,650,496 total
🎯 Initializing anchors with k-means on projected space...
  Running k-means on text projections...
  Running k-means on image projections...
✓ Anchors initialized.
✓ DataLoader ready: 112500 training samples

🚀 STARTING TRAINING: ultimate_v2_text_to_dino



/tmp/ipykernel_48/3954689281.py:463: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if device.type == "cuda" else None


Epoch 01/50:   0%|          | 0/220 [00:00<?, ?it/s]

/tmp/ipykernel_48/3954689281.py:501: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


CSLS (1/2):   0%|          | 0/5 [00:00<?, ?it/s]

...

CSLS (1/2):   0%|          | 0/5 [00:00<?, ?it/s]

CSLS (2/2):   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 06: SigLIP=0.6883 USA_T=0.0275 USA_I=0.0151 KL=5.3e+00 | LR=2.9e-04 | R@1=0.0010 R@5=0.0020 R@10=0.0038


Epoch 07/50:   0%|          | 0/220 [00:00<?, ?it/s]

KeyboardInterrupt: 

V1

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import os
import time

# -----------------------------------------------------------------
# Experiment V1: Contrastive Translator (SigLIP-style, Multi-Positive, HNM)
# -----------------------------------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------------------------------------------
# 1. Load data
# -----------------------------------------------------------------
train_data = np.load("/kaggle/input/aml-competition/train/train/train.npz")
test_data  = np.load("/kaggle/input/aml-competition/test/test/test.clean.npz")

tx_train = train_data["captions/embeddings"]  # (N_caps, 1024)
im_train = train_data["images/embeddings"]    # (N_img, 1536)
tx_test  = test_data["captions/embeddings"]   # (N_test, 1024)

# Assume 5 captions per image (Flickr30k-style)
repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)

print(f"Train shapes: tx={tx_train.shape}, im={im_train.shape}, im_expanded={im_train_expanded.shape}")
print(f"Test shape: tx_test={tx_test.shape}")

# -----------------------------------------------------------------
# 2. Preprocessing (CPU, normalization)
# -----------------------------------------------------------------
cpu_device = torch.device("cpu")

tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=cpu_device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=cpu_device)
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=cpu_device)

# Mean-center then L2-normalize (compatible with existing encoders)
tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)

tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=cpu_device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)

print("Data preprocessed and normalized (on CPU).")

# -----------------------------------------------------------------
# 3. Translator Model (Text -> Image Embedding Space)
# -----------------------------------------------------------------
class ContrastiveTranslator(nn.Module):
    """
    Text encoder that maps 1024-d text embeddings into the 1536-d image latent space.
    Uses L2 normalization and a learnable temperature (logit_scale) for similarity.
    """
    def __init__(self, input_dim=1024, hidden_dim=8192, output_dim=1536, dropout=0.4):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, output_dim),
        )

        # Learnable temperature (logit scale) as in CLIP
        # Initialize at log(1.0) => scale=1 initially
        self.logit_scale = nn.Parameter(torch.tensor(0.0))

        print(f"Initializing ContrastiveTranslator (hidden_dim={hidden_dim}, output_dim={output_dim})")

    def forward(self, x, normalize=True):
        z = self.encoder(x)
        if normalize:
            z = F.normalize(z, p=2, dim=1)
        return z

    def get_logit_scale(self):
        # optional convenience
        return self.logit_scale.exp()


# -----------------------------------------------------------------
# 4. SigLIP-style Contrastive Loss with Multi-Positives + HNM
# -----------------------------------------------------------------
# Hyperparameters for loss
HNM_K = 2              # number of hard negatives per row/column
HNM_WEIGHT = 0.5       # weight of hard negative term
LOGIT_SCALE_MIN = -4.0 # clamp range for logit_scale (exp(-4) ~ 0.018)
LOGIT_SCALE_MAX = 4.0  # exp(4) ~ 54.6

def contrastive_siglip_loss(text_emb, img_emb, img_indices, logit_scale, hnm_k=HNM_K, hnm_weight=HNM_WEIGHT):
    """
    text_emb: (B, d)
    img_emb:  (B, d)  (L2-normalized)
    img_indices: (B,) underlying image indices (same for all captions of same image)
    logit_scale: scalar tensor (exp(logit_scale_param))
    """
    B = text_emb.size(0)
    device = text_emb.device

    # Similarity matrix S_ij = scale * cos_sim(t_i, v_j)
    sims = logit_scale * (text_emb @ img_emb.T)  # (B, B)

    # Multi-positive mask: same underlying image index => positive
    img_idx = img_indices.to(device)
    pos_mask = (img_idx.unsqueeze(1) == img_idx.unsqueeze(0))  # (B, B)
    neg_mask = ~pos_mask

    # Avoid degenerate cases
    if pos_mask.sum() == 0:
        # Fallback: diagonal positives
        pos_mask = torch.eye(B, dtype=torch.bool, device=device)
        neg_mask = ~pos_mask

    # Main SigLIP-style loss (treat each pair independently)
    # Positive term: -log(sigmoid(s))
    pos_scores = sims[pos_mask]
    loss_pos = -F.logsigmoid(pos_scores).mean()

    # Negative term: -log(sigmoid(-s))
    neg_scores = sims[neg_mask]
    loss_neg = -F.logsigmoid(-neg_scores).mean()

    main_loss = loss_pos + loss_neg

    # Hard-negative mining (optional)
    if hnm_k > 0 and neg_mask.any():
        # Mask out positives for HNM
        neg_fill_value = torch.finfo(sims.dtype).min  # es: per float16 ≈ -65504
        sims_neg_only = sims.masked_fill(pos_mask, neg_fill_value)

        k_row = min(hnm_k, B - 1)
        k_col = min(hnm_k, B - 1)

        # Hardest negatives for each text (row-wise)
        hard_neg_vals_t, _ = sims_neg_only.topk(k=k_row, dim=1)  # (B, k_row)
        # Hardest negatives for each image (col-wise)
        hard_neg_vals_i, _ = sims_neg_only.topk(k=k_col, dim=0)  # (k_col, B)

        # Penalize these specifically
        loss_hnm_t = -F.logsigmoid(-hard_neg_vals_t).mean()
        loss_hnm_i = -F.logsigmoid(-hard_neg_vals_i).mean()
        loss_hnm = 0.5 * (loss_hnm_t + loss_hnm_i)
    else:
        loss_hnm = torch.tensor(0.0, device=device)

    total_loss = main_loss + hnm_weight * loss_hnm

    return total_loss, {
        "loss_pos": loss_pos.detach(),
        "loss_neg": loss_neg.detach(),
        "loss_hnm": loss_hnm.detach(),
        "main_loss": main_loss.detach()
    }

# -----------------------------------------------------------------
# 5. Validation split and DataLoader
# -----------------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)

idx_cpu = torch.randperm(N, device=cpu_device)
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

# Each caption index i corresponds to image index floor(i / 5)
# This encodes which captions belong to the same image.
img_indices_full = torch.arange(N, device=cpu_device) // 5

img_indices_train = img_indices_full[train_idx]
img_indices_val   = img_indices_full[val_idx]

tx_val_t = tx_train_t[val_idx]
tx_train_t_sub = tx_train_t[train_idx]

im_train_exp_sub = im_train_exp[train_idx]
im_val_t = im_train_exp[val_idx]  # not strictly needed, but kept for completeness

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)

print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# -----------------------------------------------------------------
# 6. Memory-Safe Recall@K (CSLS) for validation
# -----------------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, chunk_size=1024, ks=(1, 5, 10, 50), k_csls=10):
    """
    model: ContrastiveTranslator
    tx_queries: (Q, 1024) text embeddings (normalized on CPU)
    im_database: (D, 1536) image embeddings (normalized on CPU)
    query_img_indices: (Q,) int, true image index in im_database for each query
    """
    model.eval()
    device = next(model.parameters()).device
    all_correct_in_top_k = {k: [] for k in ks}

    # Compute text embeddings
    Q_embed_list = []
    for i in range(0, len(tx_queries), chunk_size):
        chunk = tx_queries[i:i+chunk_size].to(device)
        Q_embed_list.append(model(chunk))  # already L2-normalized
    Q_embed = torch.cat(Q_embed_list, dim=0)  # (Q, d)

    # Image embeddings on device
    D_embed = im_database.to(device)  # (D, d)

    # Precompute mean_knn_d for CSLS (database -> queries)
    mean_knn_d_list = []
    for i in tqdm(range(0, len(D_embed), 512), desc="CSLS (Pass 1/2)", leave=False):
        sim_T_chunk = D_embed[i:i+512] @ Q_embed.T  # (chunk, Q)
        knn_d_chunk = torch.topk(sim_T_chunk, k=k_csls, dim=1).values
        mean_knn_d_list.append(knn_d_chunk.mean(1))
    mean_knn_d = torch.cat(mean_knn_d_list, dim=0).unsqueeze(0)  # (1, D)

    gt = query_img_indices.to(device)  # (Q,)

    # CSLS similarity + R@K
    for i in tqdm(range(0, len(Q_embed), chunk_size), desc="CSLS (Pass 2/2)", leave=False):
        sim_chunk = Q_embed[i:i+chunk_size] @ D_embed.T  # (q_chunk, D)
        gt_chunk = gt[i:i+chunk_size]                    # (q_chunk,)

        knn_q_chunk = torch.topk(sim_chunk, k=k_csls, dim=1).values
        mean_knn_q = knn_q_chunk.mean(1, keepdim=True)   # (q_chunk, 1)

        csls_sim_chunk = 2 * sim_chunk - mean_knn_q - mean_knn_d  # (q_chunk, D)

        top_indices_chunk = torch.argsort(csls_sim_chunk, dim=1, descending=True)

        for k in ks:
            top_k_preds = top_indices_chunk[:, :k]
            correct_in_top_k = (top_k_preds == gt_chunk.unsqueeze(1)).any(dim=1)
            all_correct_in_top_k[k].append(correct_in_top_k)

    recalls = {}
    for k in ks:
        recalls[f"R@{k}"] = torch.cat(all_correct_in_top_k[k]).float().mean().item()
    return recalls

# -----------------------------------------------------------------
# 7. Training Loop
# -----------------------------------------------------------------
EPOCHS = 60
START_LR = 3e-4
WEIGHT_DECAY = 4e-4

RUN_ID = "V1_contrastive"
FINAL_SAVE_PATH = f"{RUN_ID}_best.pth"

# Use a subset for quick per-epoch validation
val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique  # all unique images

model = ContrastiveTranslator(
    input_dim=1024,
    hidden_dim=8192,
    output_dim=1536,
    dropout=0.4
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=START_LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.amp.GradScaler('cuda' if device.type == "cuda" else 'cpu')

print(f"\n--- Starting V1 Contrastive Translator (RUN {RUN_ID}) ---")
print(f"Training (Epochs: {EPOCHS}, LR={START_LR}, WD={WEIGHT_DECAY}, HNM_K={HNM_K}, HNM_WEIGHT={HNM_WEIGHT})...\n")

best_val_r10 = 0.0
for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0
    total_pos_loss = 0.0
    total_neg_loss = 0.0
    total_hnm_loss = 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch:03d}/{EPOCHS}", leave=False):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        img_indices = img_indices.to(device)

        optimizer.zero_grad()
        with torch.amp.autocast(device_type=device.type if device.type in ["cuda", "cpu"] else "cpu"):
            text_emb = model(x_batch)  # (B, 1536)
            img_emb = F.normalize(y_batch, p=2, dim=1)  # ensure normalized

            logit_scale = model.logit_scale.exp()
            loss, loss_dict = contrastive_siglip_loss(
                text_emb,
                img_emb,
                img_indices,
                logit_scale,
                hnm_k=HNM_K,
                hnm_weight=HNM_WEIGHT
            )

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        # Clamp logit_scale to a reasonable range
        with torch.no_grad():
            model.logit_scale.clamp_(LOGIT_SCALE_MIN, LOGIT_SCALE_MAX)

        total_loss += loss.item()
        total_pos_loss += loss_dict["loss_pos"].item()
        total_neg_loss += loss_dict["loss_neg"].item()
        total_hnm_loss += loss_dict["loss_hnm"].item()

    current_lr = scheduler.get_last_lr()[0]
    scheduler.step()

    avg_loss = total_loss / len(train_loader)
    avg_pos = total_pos_loss / len(train_loader)
    avg_neg = total_neg_loss / len(train_loader)
    avg_hnm = total_hnm_loss / len(train_loader)
    current_scale = model.logit_scale.exp().item()

    # Quick validation on subset
    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset, chunk_size=1024)
    current_r10 = rec['R@10']

    save_marker = ""
    if current_r10 > best_val_r10:
        best_val_r10 = current_r10
        torch.save(model.state_dict(), FINAL_SAVE_PATH)
        save_marker = "<- Best R@10! Saved."

    print(
        f"Epoch {epoch:03d}: "
        f"Loss={avg_loss:.4f} (Pos={avg_pos:.4f}, Neg={avg_neg:.4f}, HNM={avg_hnm:.4f}) "
        f"(LR={current_lr:.1E}, scale={current_scale:.3f}) "
        f"| R@10(ValSub)={current_r10:.4f} {save_marker}"
    )

print(f"\n🎯 Training complete. Best model (R@10={best_val_r10:.4f}) saved as {FINAL_SAVE_PATH}")

# -----------------------------------------------------------------
# 8. Inference + Submission
# -----------------------------------------------------------------
print(f"Loading BEST model from {FINAL_SAVE_PATH} for inference...")
model = ContrastiveTranslator(
    input_dim=1024,
    hidden_dim=8192,
    output_dim=1536,
    dropout=0.4
).to(device)
model.load_state_dict(torch.load(FINAL_SAVE_PATH, map_location=device))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in tqdm(range(0, len(tx_test_t), 1024), desc="Generating Sub"):
        chunk = tx_test_t[i:i+1024].to(device)
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})

submission_filename = f"submission_{RUN_ID}.csv"
submission.to_csv(submission_filename, index=False)
print(f"\n✅ {submission_filename} saved successfully.")

# -----------------------------------------------------------------
# 9. Final Validation on Full Val Set
# -----------------------------------------------------------------
print("Calculating final validation scores on full val set...")
rec_full = recall_at_k(model, tx_val_t, im_train_t_unique, img_indices_val, chunk_size=1024)
print("\n--- Full Validation Results ---")
print(f"Validation Recall@1:  {rec_full['R@1']:<8.4f}")
print(f"Validation Recall@5:  {rec_full['R@5']:<8.4f}")
print(f"Validation Recall@10: {rec_full['R@10']:<8.4f}")
print(f"Validation Recall@50: {rec_full['R@50']:<8.4f}")


Using device: cuda
Train shapes: tx=(125000, 1024), im=(25000, 1536), im_expanded=(125000, 1536)
Test shape: tx_test=(1500, 1024)
Data preprocessed and normalized (on CPU).
Train pairs: 112500, Validation pairs: 12500
Initializing ContrastiveTranslator (hidden_dim=8192, output_dim=1536)

--- Starting V1 Contrastive Translator (RUN V1_contrastive) ---
Training (Epochs: 60, LR=0.0003, WD=0.0004, HNM_K=2, HNM_WEIGHT=0.5)...



Epoch 001/60:   0%|          | 0/220 [00:00<?, ?it/s]

CSLS (Pass 1/2):   0%|          | 0/49 [00:00<?, ?it/s]

CSLS (Pass 2/2):   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 001: Loss=1.6769 (Pos=0.5401, Neg=0.6947, HNM=0.8842) (LR=3.0E-04, scale=1.078) | R@10(ValSub)=0.3746 <- Best R@10! Saved.


...

Generating Sub:   0%|          | 0/2 [00:00<?, ?it/s]


✅ submission_V1_contrastive.csv saved successfully.
Calculating final validation scores on full val set...


CSLS (Pass 1/2):   0%|          | 0/49 [00:00<?, ?it/s]

CSLS (Pass 2/2):   0%|          | 0/13 [00:00<?, ?it/s]


--- Full Validation Results ---
Validation Recall@1:  0.3402  
Validation Recall@5:  0.5616  
Validation Recall@10: 0.6513  
Validation Recall@50: 0.8043  


In [24]:
import os
import copy
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm

# -----------------------------------------------------------------
# V2: SigLIP + NDGL + HNR + USA-lite
# -----------------------------------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------------------------------------------
# 1. Load data
# -----------------------------------------------------------------

tx_train = train_data["captions/embeddings"]  # (N_caps, 1024)
im_train = train_data["images/embeddings"]    # (N_img, 1536)
tx_test = test_data["captions/embeddings"]    # (N_test, 1024)

# Assume 5 captions per image (Flickr30k-style)
repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)

print(f"Train shapes: tx={tx_train.shape}, im={im_train.shape}, im_expanded={im_train_expanded.shape}")
print(f"Test shape: tx_test={tx_test.shape}")

# -----------------------------------------------------------------
# 2. Preprocessing (CPU, normalization)
# -----------------------------------------------------------------
cpu_device = torch.device("cpu")

tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=cpu_device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=cpu_device)
tx_test_t = torch.as_tensor(tx_test, dtype=torch.float32, device=cpu_device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)

tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=cpu_device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)

print("Data preprocessed and normalized (on CPU).")

# -----------------------------------------------------------------
# 3. Translator Model (Text -> Image Embedding Space)
# -----------------------------------------------------------------
class ContrastiveTranslator(nn.Module):
    """
    Text encoder that maps 1024-d text embeddings into the 1536-d image latent space.
    Uses L2 normalization and a learnable temperature (logit_scale) for similarity.
    """
    def __init__(self, input_dim=1024, hidden_dim=8192, output_dim=1536, dropout=0.4):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, output_dim),
        )

        # Learnable temperature as in CLIP
        self.logit_scale = nn.Parameter(torch.tensor(0.0))

        print(f"Initializing ContrastiveTranslator (hidden_dim={hidden_dim}, output_dim={output_dim})")

    def forward(self, x, normalize=True):
        z = self.encoder(x)
        if normalize:
            z = F.normalize(z, p=2, dim=1)
        return z

    def get_logit_scale(self):
        return self.logit_scale.exp()


# -----------------------------------------------------------------
# 4. V2 Loss: SigLIP + NDGL + HNR + USA-lite
# -----------------------------------------------------------------

# Hyperparameters
LAMBDA_NEG = 0.5          # weight for negative term in SigLIP core
LAMBDA_NDGL = 0.05        # weight for NDGL (KL over negatives)
LAMBDA_HNR = 0.01         # weight for HNR hinge
LAMBDA_INTRA = 0.02       # weight for intra-text alignment
NDGL_TEMPERATURE = 0.2    # temperature for NDGL softmax
NDGL_START_EPOCH = 5      # start using NDGL/HNR from this epoch
EMA_MOMENTUM = 0.99       # EMA teacher momentum
LOGIT_SCALE_MIN = -2.0    # clamp logit_scale (exp(-2)=0.135)
LOGIT_SCALE_MAX = 2.5     # (exp(2.5)≈12.18)
HNR_MARGIN = 0.01         # gamma in HNR

def v2_loss(
    text_emb,
    img_emb,
    img_indices,
    student_logit_scale,
    teacher_model=None,
    x_batch_for_teacher=None,
    lambda_neg=LAMBDA_NEG,
    lambda_ndgl=0.0,
    lambda_hnr=0.0,
    lambda_intra=0.0,
    ndgl_temperature=NDGL_TEMPERATURE
):
    """
    V2 loss = SigLIP core + NDGL (row/col KL) + HNR + intra-text alignment (USA-lite).

    text_emb: (B, d)
    img_emb:  (B, d)  (L2-normalized)
    img_indices: (B,) ints for image ids (multi-positive groups)
    student_logit_scale: scalar tensor (exp(logit_scale))
    teacher_model: EMA teacher (ContrastiveTranslator) or None
    x_batch_for_teacher: original text batch for teacher forward
    """
    B = text_emb.size(0)
    device = text_emb.device
    eps = 1e-8

    # -------------------------------
    # Core SigLIP: binary logistic over all pairs
    # -------------------------------
    sims_student = student_logit_scale * (text_emb @ img_emb.T)  # (B, B)

    img_idx = img_indices.to(device)
    pos_mask = (img_idx.unsqueeze(1) == img_idx.unsqueeze(0))  # (B, B)
    neg_mask = ~pos_mask

    if pos_mask.sum() == 0:
        pos_mask = torch.eye(B, dtype=torch.bool, device=device)
        neg_mask = ~pos_mask

    pos_scores = sims_student[pos_mask]
    neg_scores = sims_student[neg_mask]

    loss_pos = -F.logsigmoid(pos_scores).mean()
    loss_neg = -F.logsigmoid(-neg_scores).mean()

    loss_main = loss_pos + lambda_neg * loss_neg

    # -------------------------------
    # USA-lite: intra-text alignment
    # same-image captions should be close in translator space
    # -------------------------------
    loss_intra = torch.tensor(0.0, device=device)
    if lambda_intra > 0.0:
        same_img = pos_mask.triu(diagonal=1)  # upper triangle (avoid duplicates / self)
        idx_i, idx_j = same_img.nonzero(as_tuple=True)
        if idx_i.numel() > 0:
            zi = text_emb[idx_i]
            zj = text_emb[idx_j]
            loss_intra = ((zi - zj) ** 2).sum(dim=1).mean()

    # -------------------------------
    # NDGL + HNR via EMA teacher (optional)
    # -------------------------------
    loss_ndgl = torch.tensor(0.0, device=device)
    loss_hnr = torch.tensor(0.0, device=device)

    if (lambda_ndgl > 0.0 or lambda_hnr > 0.0) and teacher_model is not None and x_batch_for_teacher is not None:
        with torch.no_grad():
            teacher_model.eval()
            teacher_text_emb = teacher_model(x_batch_for_teacher)         # (B, d)
            teacher_img_emb = F.normalize(img_emb, p=2, dim=1)            # same images
            teacher_scale = teacher_model.get_logit_scale()
            sims_teacher = teacher_scale * (teacher_text_emb @ teacher_img_emb.T)

        neg_fill_student = torch.finfo(sims_student.dtype).min
        neg_fill_teacher = torch.finfo(sims_teacher.dtype).min

        # mask positives out for NDGL/HNR
        sims_student_neg = sims_student.masked_fill(pos_mask, neg_fill_student)
        sims_teacher_neg = sims_teacher.masked_fill(pos_mask, neg_fill_teacher)

        # ---- NDGL: KL over negative distributions (rows + cols) ----
        if lambda_ndgl > 0.0:
            # Row-wise (text as query)
            p_student_row = F.softmax(sims_student_neg / ndgl_temperature, dim=1)
            p_teacher_row = F.softmax(sims_teacher_neg / ndgl_temperature, dim=1)

            kl_row = (p_teacher_row * (torch.log(p_teacher_row + eps) - torch.log(p_student_row + eps))).sum(dim=1).mean()

            # Column-wise (image as query)
            sims_student_neg_T = sims_student_neg.T
            sims_teacher_neg_T = sims_teacher_neg.T

            p_student_col = F.softmax(sims_student_neg_T / ndgl_temperature, dim=1)
            p_teacher_col = F.softmax(sims_teacher_neg_T / ndgl_temperature, dim=1)

            kl_col = (p_teacher_col * (torch.log(p_teacher_col + eps) - torch.log(p_student_col + eps))).sum(dim=1).mean()

            loss_ndgl = 0.5 * (kl_row + kl_col)

        # ---- HNR: align hardest negatives from teacher/student ----
        if lambda_hnr > 0.0:
            # Row-wise (text as query)
            # argmax over negatives only (positives are -inf-ish)
            j_hat = sims_student_neg.argmax(dim=1)   # student's hardest negative per row
            j_tilde = sims_teacher_neg.argmax(dim=1) # teacher's hardest negative per row

            s_row_hat = sims_student[torch.arange(B, device=device), j_hat]
            s_row_tilde = sims_student[torch.arange(B, device=device), j_tilde]

            gap_row = (s_row_hat - HNR_MARGIN) - s_row_tilde
            loss_hnr_row = F.relu(gap_row).mean()

            # Column-wise (image as query)
            i_hat = sims_student_neg.argmax(dim=0)   # student's hardest per column
            i_tilde = sims_teacher_neg.argmax(dim=0) # teacher's hardest per column

            s_col_hat = sims_student[i_hat, torch.arange(B, device=device)]
            s_col_tilde = sims_student[i_tilde, torch.arange(B, device=device)]

            gap_col = (s_col_hat - HNR_MARGIN) - s_col_tilde
            loss_hnr_col = F.relu(gap_col).mean()

            loss_hnr = 0.5 * (loss_hnr_row + loss_hnr_col)

    # -------------------------------
    # Total loss
    # -------------------------------
    total_loss = loss_main \
                 + lambda_ndgl * loss_ndgl \
                 + lambda_hnr * loss_hnr \
                 + lambda_intra * loss_intra

    loss_dict = {
        "loss_main": loss_main.detach(),
        "loss_pos": loss_pos.detach(),
        "loss_neg": loss_neg.detach(),
        "loss_intra": loss_intra.detach(),
        "loss_ndgl": loss_ndgl.detach(),
        "loss_hnr": loss_hnr.detach(),
    }

    return total_loss, loss_dict


# -----------------------------------------------------------------
# 5. Validation split and DataLoader
# -----------------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)

idx_cpu = torch.randperm(N, device=cpu_device)
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

# Each caption index i -> image index floor(i/5)
img_indices_full = torch.arange(N, device=cpu_device) // 5

img_indices_train = img_indices_full[train_idx]
img_indices_val = img_indices_full[val_idx]

tx_val_t = tx_train_t[val_idx]
tx_train_t_sub = tx_train_t[train_idx]
im_train_exp_sub = im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)

print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# -----------------------------------------------------------------
# 6. Memory-Safe Recall@K (CSLS)
# -----------------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, chunk_size=1024, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    device = next(model.parameters()).device
    all_correct_in_top_k = {k: [] for k in ks}

    Q_embed_list = []
    for i in range(0, len(tx_queries), chunk_size):
        chunk = tx_queries[i:i+chunk_size].to(device)
        Q_embed_list.append(model(chunk))
    Q_embed = torch.cat(Q_embed_list, dim=0)

    D_embed = im_database.to(device)

    # CSLS pass 1
    mean_knn_d_list = []
    for i in tqdm(range(0, len(D_embed), 512), desc="CSLS (Pass 1/2)", leave=False):
        sim_T_chunk = D_embed[i:i+512] @ Q_embed.T
        knn_d_chunk = torch.topk(sim_T_chunk, k=k_csls, dim=1).values
        mean_knn_d_list.append(knn_d_chunk.mean(1))
    mean_knn_d = torch.cat(mean_knn_d_list, dim=0).unsqueeze(0)

    gt = query_img_indices.to(device)

    # CSLS pass 2
    for i in tqdm(range(0, len(Q_embed), chunk_size), desc="CSLS (Pass 2/2)", leave=False):
        sim_chunk = Q_embed[i:i+chunk_size] @ D_embed.T
        gt_chunk = gt[i:i+chunk_size]

        knn_q_chunk = torch.topk(sim_chunk, k=k_csls, dim=1).values
        mean_knn_q = knn_q_chunk.mean(1, keepdim=True)

        csls_sim_chunk = 2 * sim_chunk - mean_knn_q - mean_knn_d
        top_indices_chunk = torch.argsort(csls_sim_chunk, dim=1, descending=True)

        for k in ks:
            top_k_preds = top_indices_chunk[:, :k]
            correct_in_top_k = (top_k_preds == gt_chunk.unsqueeze(1)).any(dim=1)
            all_correct_in_top_k[k].append(correct_in_top_k)

    recalls = {}
    for k in ks:
        recalls[f"R@{k}"] = torch.cat(all_correct_in_top_k[k]).float().mean().item()
    return recalls

# -----------------------------------------------------------------
# 7. Training Loop (with EMA teacher & V2 loss)
# -----------------------------------------------------------------
EPOCHS = 60
START_LR = 3e-4
WEIGHT_DECAY = 4e-4

RUN_ID = "V2_contrastive"
FINAL_SAVE_PATH = f"{RUN_ID}_best.pth"

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

model = ContrastiveTranslator(
    input_dim=1024,
    hidden_dim=8192,
    output_dim=1536,
    dropout=0.4
).to(device)

# EMA teacher
teacher_model = copy.deepcopy(model).to(device)
for p in teacher_model.parameters():
    p.requires_grad = False

optimizer = torch.optim.AdamW(model.parameters(), lr=START_LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.amp.GradScaler()

print(f"\n--- Starting V2 Contrastive Translator (RUN {RUN_ID}) ---")
print(
    f"Training (Epochs: {EPOCHS}, LR={START_LR}, WD={WEIGHT_DECAY}, "
    f"L_NEG={LAMBDA_NEG}, L_NDGL={LAMBDA_NDGL}, L_HNR={LAMBDA_HNR}, L_INTRA={LAMBDA_INTRA})...\n"
)

best_val_r10 = 0.0

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0
    total_pos = 0.0
    total_neg = 0.0
    total_ndgl = 0.0
    total_hnr = 0.0
    total_intra = 0.0

    enable_dist = epoch >= NDGL_START_EPOCH
    lambda_ndgl = LAMBDA_NDGL if enable_dist else 0.0
    lambda_hnr = LAMBDA_HNR if enable_dist else 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch:03d}/{EPOCHS}", leave=False):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        img_indices = img_indices.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type=device.type if device.type in ["cuda", "cpu"] else "cpu"):
            text_emb = model(x_batch)               # (B, 1536)
            img_emb = F.normalize(y_batch, p=2, dim=1)

            logit_scale_student = model.get_logit_scale()

            loss, loss_dict = v2_loss(
                text_emb=text_emb,
                img_emb=img_emb,
                img_indices=img_indices,
                student_logit_scale=logit_scale_student,
                teacher_model=teacher_model if enable_dist else None,
                x_batch_for_teacher=x_batch if enable_dist else None,
                lambda_neg=LAMBDA_NEG,
                lambda_ndgl=lambda_ndgl,
                lambda_hnr=lambda_hnr,
                lambda_intra=LAMBDA_INTRA,
                ndgl_temperature=NDGL_TEMPERATURE
            )

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        # EMA update for teacher
        with torch.no_grad():
            for p_t, p_s in zip(teacher_model.parameters(), model.parameters()):
                p_t.data.mul_(EMA_MOMENTUM).add_(p_s.data, alpha=1.0 - EMA_MOMENTUM)

            # Clamp student logit_scale to stable range
            model.logit_scale.clamp_(LOGIT_SCALE_MIN, LOGIT_SCALE_MAX)

        total_loss += loss.item()
        total_pos += loss_dict["loss_pos"].item()
        total_neg += loss_dict["loss_neg"].item()
        total_intra += loss_dict["loss_intra"].item()
        total_ndgl += loss_dict["loss_ndgl"].item()
        total_hnr += loss_dict["loss_hnr"].item()

    current_lr = scheduler.get_last_lr()[0]
    scheduler.step()

    num_batches = len(train_loader)
    avg_loss = total_loss / num_batches
    avg_pos = total_pos / num_batches
    avg_neg = total_neg / num_batches
    avg_intra = total_intra / num_batches
    avg_ndgl = total_ndgl / num_batches
    avg_hnr = total_hnr / num_batches
    current_scale = model.get_logit_scale().item()

    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset, chunk_size=1024)
    current_r10 = rec['R@10']

    save_marker = ""
    if current_r10 > best_val_r10:
        best_val_r10 = current_r10
        torch.save(model.state_dict(), FINAL_SAVE_PATH)
        save_marker = "<- Best R@10! Saved."

    print(
        f"Epoch {epoch:03d}: "
        f"Loss={avg_loss:.4f} (Pos={avg_pos:.4f}, Neg={avg_neg:.4f}, Intra={avg_intra:.4f}, "
        f"NDGL={avg_ndgl:.4f}, HNR={avg_hnr:.4f}) "
        f"(LR={current_lr:.1E}, scale={current_scale:.3f}, NDGL_on={enable_dist}) "
        f"| R@10(ValSub)={current_r10:.4f} {save_marker}"
    )

print(f"\n🎯 Training complete. Best model (R@10={best_val_r10:.4f}) saved as {FINAL_SAVE_PATH}")

# -----------------------------------------------------------------
# 8. Inference + Submission
# -----------------------------------------------------------------
print(f"Loading BEST model from {FINAL_SAVE_PATH} for inference...")
model = ContrastiveTranslator(
    input_dim=1024,
    hidden_dim=8192,
    output_dim=1536,
    dropout=0.4
).to(device)
model.load_state_dict(torch.load(FINAL_SAVE_PATH, map_location=device))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in tqdm(range(0, len(tx_test_t), 1024), desc="Generating Sub"):
        chunk = tx_test_t[i:i+1024].to(device)
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})

submission_filename = f"submission_{RUN_ID}.csv"
submission.to_csv(submission_filename, index=False)
print(f"\n✅ {submission_filename} saved successfully.")

# -----------------------------------------------------------------
# 9. Final Validation on Full Val Set
# -----------------------------------------------------------------
print("Calculating final validation scores on full val set...")
rec_full = recall_at_k(model, tx_val_t, im_train_t_unique, img_indices_val, chunk_size=1024)
print("\n--- Full Validation Results ---")
print(f"Validation Recall@1:  {rec_full['R@1']:<8.4f}")
print(f"Validation Recall@5:  {rec_full['R@5']:<8.4f}")
print(f"Validation Recall@10: {rec_full['R@10']:<8.4f}")
print(f"Validation Recall@50: {rec_full['R@50']:<8.4f}")


Using device: cuda
Train shapes: tx=(125000, 1024), im=(25000, 1536), im_expanded=(125000, 1536)
Test shape: tx_test=(1500, 1024)
Data preprocessed and normalized (on CPU).
Train pairs: 112500, Validation pairs: 12500
Initializing ContrastiveTranslator (hidden_dim=8192, output_dim=1536)

--- Starting V2 Contrastive Translator (RUN V2_contrastive) ---
Training (Epochs: 60, LR=0.0003, WD=0.0004, L_NEG=0.5, L_NDGL=0.05, L_HNR=0.01, L_INTRA=0.02)...



Epoch 001/60:   0%|          | 0/220 [00:00<?, ?it/s]

CSLS (Pass 1/2):   0%|          | 0/49 [00:00<?, ?it/s]

CSLS (Pass 2/2):   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 001: Loss=0.8804 (Pos=0.5200, Neg=0.6974, Intra=0.5868, NDGL=0.0000, HNR=0.0000) (LR=3.0E-04, scale=1.073, NDGL_on=False) | R@10(ValSub)=0.1440 <- Best R@10! Saved.


...

Generating Sub:   0%|          | 0/2 [00:00<?, ?it/s]


✅ submission_V2_contrastive.csv saved successfully.
Calculating final validation scores on full val set...


CSLS (Pass 1/2):   0%|          | 0/49 [00:00<?, ?it/s]

CSLS (Pass 2/2):   0%|          | 0/13 [00:00<?, ?it/s]


--- Full Validation Results ---
Validation Recall@1:  0.1294  
Validation Recall@5:  0.2986  
Validation Recall@10: 0.3962  
Validation Recall@50: 0.6540  


In [30]:
import os
import copy
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm

# -----------------------------------------------------------------
# V3: Flow-lite Translator + SigLIP + CUSA-style soft labels + light NDGL/HNR
# -----------------------------------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------------------------------------------
# 1. Load data
# -----------------------------------------------------------------
tx_train = train_data["captions/embeddings"]  # (N_caps, 1024)
im_train = train_data["images/embeddings"]    # (N_img, 1536)
tx_test = test_data["captions/embeddings"]    # (N_test, 1024)

# Assume 5 captions per image (Flickr30k-style)
repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)

print(f"Train shapes: tx={tx_train.shape}, im={im_train.shape}, im_expanded={im_train_expanded.shape}")
print(f"Test shape: tx_test={tx_test.shape}")

# -----------------------------------------------------------------
# 2. Preprocessing (CPU, normalization)
# -----------------------------------------------------------------
cpu_device = torch.device("cpu")

tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=cpu_device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=cpu_device)
tx_test_t = torch.as_tensor(tx_test, dtype=torch.float32, device=cpu_device)

# Mean-center then L2-normalize (as before)
tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)

tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=cpu_device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)

print("Data preprocessed and normalized (on CPU).")

# -----------------------------------------------------------------
# 3. Flow-lite Translator (Text -> Image Embedding Space)
# -----------------------------------------------------------------
class FlowBlock(nn.Module):
    """
    Simple residual MLP block: x -> x + f(x)
    Used as a discrete flow step in the latent (image) space.
    """
    def __init__(self, dim, hidden_dim, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
        )

    def forward(self, x):
        return x + self.net(x)


class FlowTranslatorV3(nn.Module):
    """
    Text encoder that:
      - projects 1024-d text embeddings into 1536-d image latent
      - refines via K residual FlowBlocks
      - normalizes output and uses a learnable logit_scale
    """
    def __init__(
        self,
        input_dim=1024,
        latent_dim=1536,
        flow_hidden_dim=4096,
        flow_steps=4,
        dropout=0.2,
    ):
        super().__init__()
        self.latent_dim = latent_dim

        self.text_proj = nn.Sequential(
            nn.Linear(input_dim, flow_hidden_dim),
            nn.GELU(),
            nn.LayerNorm(flow_hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(flow_hidden_dim, latent_dim),
        )

        self.flow_blocks = nn.ModuleList(
            [FlowBlock(latent_dim, flow_hidden_dim, dropout=dropout) for _ in range(flow_steps)]
        )

        # Learnable temperature as in CLIP
        self.logit_scale = nn.Parameter(torch.tensor(0.0))

        print(f"Initializing FlowTranslatorV3 (flow_steps={flow_steps}, latent_dim={latent_dim}, hidden={flow_hidden_dim})")

    def forward(self, x, normalize=True):
        # x: (B, input_dim) in base text space (already normalized)
        z = self.text_proj(x)
        for block in self.flow_blocks:
            z = block(z)
        if normalize:
            z = F.normalize(z, p=2, dim=1)
        return z

    def get_logit_scale(self):
        return self.logit_scale.exp()


# -----------------------------------------------------------------
# 4. V3 Loss: SigLIP core + CUSA (CSA/USA) + light NDGL + light HNR
# -----------------------------------------------------------------

# Hyperparameters
LAMBDA_NEG = 1.0          # stronger push on negatives vs V2
LAMBDA_CSA = 0.1          # cross-modal soft labels (CUSA-CSA)
LAMBDA_USA = 0.05         # intra-text soft labels (CUSA-USA)
LAMBDA_NDGL = 0.01        # very light NDGL
LAMBDA_HNR = 0.003        # very light HNR
NDGL_TEMPERATURE = 0.3    # softmax temp for NDGL
NDGL_START_EPOCH = 10     # start NDGL/HNR later
EMA_MOMENTUM = 0.99       # EMA teacher momentum
LOGIT_SCALE_MIN = -1.5    # clamp for logit_scale parameter (exp(-1.5)=0.22)
LOGIT_SCALE_MAX = 2.5     # (exp(2.5)=12.18)
HNR_MARGIN = 0.01         # gamma in HNR
EPS = 1e-8

def v3_loss(
    text_emb,
    img_emb,
    img_indices,
    logit_scale_student,
    tx_base_batch,
    im_base_batch,          # rimane per compatibilità, ma non usato in CSA
    teacher_model=None,
    x_batch_for_teacher=None,
    lambda_neg=LAMBDA_NEG,
    lambda_csa=LAMBDA_CSA,  # verrà ignorato (CSA = 0)
    lambda_usa=LAMBDA_USA,
    lambda_ndgl=0.0,
    lambda_hnr=0.0,
):
    """
    V3 loss:
      - core SigLIP (binary logistic) con multi-positives
      - USA (text): student text-text vs base text-text
      - NDGL: KL tra distribuzioni negative teacher/student (light)
      - HNR: hinge sulla hardest negative (light)

    NOTA: per semplicità e stabilità, la parte CSA (cross-modal soft labels
    tra base text e base image) è disattivata: loss_csa = 0.
    """
    device = text_emb.device
    B = text_emb.size(0)

    # Base text features (1024-d) sul device
    tx_base = tx_base_batch.to(device)  # (B, 1024)

    # ------------------------------------------------------------------
    # Core SigLIP loss (student text_emb vs img_emb)
    # ------------------------------------------------------------------
    sims_student = logit_scale_student * (text_emb @ img_emb.T)  # (B, B)

    img_idx = img_indices.to(device)
    pos_mask = (img_idx.unsqueeze(1) == img_idx.unsqueeze(0))  # (B, B)
    neg_mask = ~pos_mask

    if pos_mask.sum() == 0:
        pos_mask = torch.eye(B, dtype=torch.bool, device=device)
        neg_mask = ~pos_mask

    pos_scores = sims_student[pos_mask]
    neg_scores = sims_student[neg_mask]

    loss_pos = -F.logsigmoid(pos_scores).mean()
    loss_neg = -F.logsigmoid(-neg_scores).mean()
    loss_main = loss_pos + lambda_neg * loss_neg

    # ------------------------------------------------------------------
    # USA (text): Uni-modal soft-label alignment (text-text)
    # base text-text cosine vs student text-text cosine
    # ------------------------------------------------------------------
    with torch.no_grad():
        base_tt = tx_base @ tx_base.T  # (B, B) in [-1, 1]

    student_tt = text_emb @ text_emb.T  # (B, B) in [-1, 1]
    loss_usa = F.mse_loss(student_tt, base_tt)

    # ------------------------------------------------------------------
    # CSA disattivata per questo V3 (shape-mismatch su base text/image)
    # ------------------------------------------------------------------
    loss_csa = torch.tensor(0.0, device=device)

    # ------------------------------------------------------------------
    # NDGL + HNR via EMA teacher (optional, light)
    # ------------------------------------------------------------------
    loss_ndgl = torch.tensor(0.0, device=device)
    loss_hnr = torch.tensor(0.0, device=device)

    if (lambda_ndgl > 0.0 or lambda_hnr > 0.0) and teacher_model is not None and x_batch_for_teacher is not None:
        with torch.no_grad():
            teacher_model.eval()
            teacher_text_emb = teacher_model(x_batch_for_teacher)             # (B, d)
            teacher_img_emb = F.normalize(img_emb, p=2, dim=1)                # same image batch
            logit_scale_teacher = teacher_model.get_logit_scale()
            sims_teacher = logit_scale_teacher * (teacher_text_emb @ teacher_img_emb.T)

        neg_fill_student = torch.finfo(sims_student.dtype).min
        neg_fill_teacher = torch.finfo(sims_teacher.dtype).min

        sims_student_neg = sims_student.masked_fill(pos_mask, neg_fill_student)
        sims_teacher_neg = sims_teacher.masked_fill(pos_mask, neg_fill_teacher)

        # NDGL: KL on negative distributions
        if lambda_ndgl > 0.0:
            p_st_row = F.softmax(sims_student_neg / NDGL_TEMPERATURE, dim=1)
            p_te_row = F.softmax(sims_teacher_neg / NDGL_TEMPERATURE, dim=1)
            kl_row = (p_te_row * (torch.log(p_te_row + EPS) - torch.log(p_st_row + EPS))).sum(dim=1).mean()

            sims_student_neg_T = sims_student_neg.T
            sims_teacher_neg_T = sims_teacher_neg.T
            p_st_col = F.softmax(sims_student_neg_T / NDGL_TEMPERATURE, dim=1)
            p_te_col = F.softmax(sims_teacher_neg_T / NDGL_TEMPERATURE, dim=1)
            kl_col = (p_te_col * (torch.log(p_te_col + EPS) - torch.log(p_st_col + EPS))).sum(dim=1).mean()

            loss_ndgl = 0.5 * (kl_row + kl_col)

        # HNR: hardest negative rectification
        if lambda_hnr > 0.0:
            # row-wise
            j_hat = sims_student_neg.argmax(dim=1)
            j_tilde = sims_teacher_neg.argmax(dim=1)
            s_row_hat = sims_student[torch.arange(B, device=device), j_hat]
            s_row_tilde = sims_student[torch.arange(B, device=device), j_tilde]
            gap_row = (s_row_hat - HNR_MARGIN) - s_row_tilde
            loss_hnr_row = F.relu(gap_row).mean()

            # col-wise
            i_hat = sims_student_neg.argmax(dim=0)
            i_tilde = sims_teacher_neg.argmax(dim=0)
            s_col_hat = sims_student[i_hat, torch.arange(B, device=device)]
            s_col_tilde = sims_student[i_tilde, torch.arange(B, device=device)]
            gap_col = (s_col_hat - HNR_MARGIN) - s_col_tilde
            loss_hnr_col = F.relu(gap_col).mean()

            loss_hnr = 0.5 * (loss_hnr_row + loss_hnr_col)

    # ------------------------------------------------------------------
    # Total loss
    # ------------------------------------------------------------------
    total_loss = (
        loss_main
        + lambda_usa * loss_usa
        + lambda_ndgl * loss_ndgl
        + lambda_hnr * loss_hnr
        # λ_csa * loss_csa è zero, ma lo teniamo per compatibilità
    )

    loss_dict = {
        "loss_main": loss_main.detach(),
        "loss_pos": loss_pos.detach(),
        "loss_neg": loss_neg.detach(),
        "loss_csa": loss_csa.detach(),
        "loss_usa": loss_usa.detach(),
        "loss_ndgl": loss_ndgl.detach(),
        "loss_hnr": loss_hnr.detach(),
    }

    return total_loss, loss_dict

# -----------------------------------------------------------------
# 5. Validation split and DataLoader
# -----------------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)

idx_cpu = torch.randperm(N, device=cpu_device)
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

# Each caption index i -> image index floor(i/5)
img_indices_full = torch.arange(N, device=cpu_device) // 5

img_indices_train = img_indices_full[train_idx]
img_indices_val = img_indices_full[val_idx]

tx_val_t = tx_train_t[val_idx]
tx_train_t_sub = tx_train_t[train_idx]
im_train_exp_sub = im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)

print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# -----------------------------------------------------------------
# 6. Memory-Safe Recall@K (CSLS)
# -----------------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, chunk_size=1024, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    device = next(model.parameters()).device
    all_correct_in_top_k = {k: [] for k in ks}

    Q_embed_list = []
    for i in range(0, len(tx_queries), chunk_size):
        chunk = tx_queries[i:i+chunk_size].to(device)
        Q_embed_list.append(model(chunk))
    Q_embed = torch.cat(Q_embed_list, dim=0)

    D_embed = im_database.to(device)

    # CSLS pass 1
    mean_knn_d_list = []
    for i in tqdm(range(0, len(D_embed), 512), desc="CSLS (Pass 1/2)", leave=False):
        sim_T_chunk = D_embed[i:i+512] @ Q_embed.T
        knn_d_chunk = torch.topk(sim_T_chunk, k=k_csls, dim=1).values
        mean_knn_d_list.append(knn_d_chunk.mean(1))
    mean_knn_d = torch.cat(mean_knn_d_list, dim=0).unsqueeze(0)

    gt = query_img_indices.to(device)

    # CSLS pass 2
    for i in tqdm(range(0, len(Q_embed), chunk_size), desc="CSLS (Pass 2/2)", leave=False):
        sim_chunk = Q_embed[i:i+chunk_size] @ D_embed.T
        gt_chunk = gt[i:i+chunk_size]

        knn_q_chunk = torch.topk(sim_chunk, k=k_csls, dim=1).values
        mean_knn_q = knn_q_chunk.mean(1, keepdim=True)

        csls_sim_chunk = 2 * sim_chunk - mean_knn_q - mean_knn_d
        top_indices_chunk = torch.argsort(csls_sim_chunk, dim=1, descending=True)

        for k in ks:
            top_k_preds = top_indices_chunk[:, :k]
            correct_in_top_k = (top_k_preds == gt_chunk.unsqueeze(1)).any(dim=1)
            all_correct_in_top_k[k].append(correct_in_top_k)

    recalls = {}
    for k in ks:
        recalls[f"R@{k}"] = torch.cat(all_correct_in_top_k[k]).float().mean().item()
    return recalls

# -----------------------------------------------------------------
# 7. Training Loop (with EMA teacher & V3 loss)
# -----------------------------------------------------------------
EPOCHS = 60
START_LR = 3e-4
WEIGHT_DECAY = 4e-4

RUN_ID = "V3_flow_cusa"
FINAL_SAVE_PATH = f"{RUN_ID}_best.pth"

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

model = FlowTranslatorV3(
    input_dim=1024,
    latent_dim=1536,
    flow_hidden_dim=4096,
    flow_steps=4,
    dropout=0.2,
).to(device)

# EMA teacher
teacher_model = copy.deepcopy(model).to(device)
for p in teacher_model.parameters():
    p.requires_grad = False

optimizer = torch.optim.AdamW(model.parameters(), lr=START_LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.amp.GradScaler()

print(f"\n--- Starting V3 Flow+CUSA Translator (RUN {RUN_ID}) ---")
print(
    f"Training (Epochs: {EPOCHS}, LR={START_LR}, WD={WEIGHT_DECAY}, "
    f"L_NEG={LAMBDA_NEG}, L_CSA={LAMBDA_CSA}, L_USA={LAMBDA_USA}, "
    f"L_NDGL={LAMBDA_NDGL}, L_HNR={LAMBDA_HNR})...\n"
)

best_val_r10 = 0.0

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0
    total_pos = 0.0
    total_neg = 0.0
    total_csa = 0.0
    total_usa = 0.0
    total_ndgl = 0.0
    total_hnr = 0.0

    enable_dist = epoch >= NDGL_START_EPOCH
    lambda_ndgl = LAMBDA_NDGL if enable_dist else 0.0
    lambda_hnr = LAMBDA_HNR if enable_dist else 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch:03d}/{EPOCHS}", leave=False):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        img_indices = img_indices.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type=device.type if device.type in ["cuda", "cpu"] else "cpu"):
            text_emb = model(x_batch)              # (B, 1536)
            img_emb = F.normalize(y_batch, p=2, dim=1)

            logit_scale_student = model.get_logit_scale()

            loss, loss_dict = v3_loss(
                text_emb=text_emb,
                img_emb=img_emb,
                img_indices=img_indices,
                logit_scale_student=logit_scale_student,
                tx_base_batch=x_batch,
                im_base_batch=y_batch,
                teacher_model=teacher_model if enable_dist else None,
                x_batch_for_teacher=x_batch if enable_dist else None,
                lambda_neg=LAMBDA_NEG,
                lambda_csa=LAMBDA_CSA,
                lambda_usa=LAMBDA_USA,
                lambda_ndgl=lambda_ndgl,
                lambda_hnr=lambda_hnr,
            )

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        # EMA & logit_scale clamp
        with torch.no_grad():
            for p_t, p_s in zip(teacher_model.parameters(), model.parameters()):
                p_t.data.mul_(EMA_MOMENTUM).add_(p_s.data, alpha=1.0 - EMA_MOMENTUM)

            model.logit_scale.clamp_(LOGIT_SCALE_MIN, LOGIT_SCALE_MAX)

        total_loss += loss.item()
        total_pos += loss_dict["loss_pos"].item()
        total_neg += loss_dict["loss_neg"].item()
        total_csa += loss_dict["loss_csa"].item()
        total_usa += loss_dict["loss_usa"].item()
        total_ndgl += loss_dict["loss_ndgl"].item()
        total_hnr += loss_dict["loss_hnr"].item()

    current_lr = scheduler.get_last_lr()[0]
    scheduler.step()

    num_batches = len(train_loader)
    avg_loss = total_loss / num_batches
    avg_pos = total_pos / num_batches
    avg_neg = total_neg / num_batches
    avg_csa = total_csa / num_batches
    avg_usa = total_usa / num_batches
    avg_ndgl = total_ndgl / num_batches
    avg_hnr = total_hnr / num_batches
    current_scale = model.get_logit_scale().item()

    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset, chunk_size=1024)
    current_r10 = rec['R@10']

    save_marker = ""
    if current_r10 > best_val_r10:
        best_val_r10 = current_r10
        torch.save(model.state_dict(), FINAL_SAVE_PATH)
        save_marker = "<- Best R@10! Saved."

    print(
        f"Epoch {epoch:03d}: "
        f"Loss={avg_loss:.4f} (Pos={avg_pos:.4f}, Neg={avg_neg:.4f}, "
        f"CSA={avg_csa:.4f}, USA={avg_usa:.4f}, NDGL={avg_ndgl:.4f}, HNR={avg_hnr:.4f}) "
        f"(LR={current_lr:.1E}, scale={current_scale:.3f}, NDGL_on={enable_dist}) "
        f"| R@10(ValSub)={current_r10:.4f} {save_marker}"
    )

print(f"\n🎯 Training complete. Best model (R@10={best_val_r10:.4f}) saved as {FINAL_SAVE_PATH}")

# -----------------------------------------------------------------
# 8. Inference + Submission
# -----------------------------------------------------------------
print(f"Loading BEST model from {FINAL_SAVE_PATH} for inference...")
model = FlowTranslatorV3(
    input_dim=1024,
    latent_dim=1536,
    flow_hidden_dim=4096,
    flow_steps=4,
    dropout=0.2,
).to(device)
model.load_state_dict(torch.load(FINAL_SAVE_PATH, map_location=device))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in tqdm(range(0, len(tx_test_t), 1024), desc="Generating Sub"):
        chunk = tx_test_t[i:i+1024].to(device)
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})

submission_filename = f"submission_{RUN_ID}.csv"
submission.to_csv(submission_filename, index=False)
print(f"\n✅ {submission_filename} saved successfully.")

# -----------------------------------------------------------------
# 9. Final Validation on Full Val Set
# -----------------------------------------------------------------
print("Calculating final validation scores on full val set...")
rec_full = recall_at_k(model, tx_val_t, im_train_t_unique, img_indices_val, chunk_size=1024)
print("\n--- Full Validation Results ---")
print(f"Validation Recall@1:  {rec_full['R@1']:<8.4f}")
print(f"Validation Recall@5:  {rec_full['R@5']:<8.4f}")
print(f"Validation Recall@10: {rec_full['R@10']:<8.4f}")
print(f"Validation Recall@50: {rec_full['R@50']:<8.4f}")


Using device: cuda
Train shapes: tx=(125000, 1024), im=(25000, 1536), im_expanded=(125000, 1536)
Test shape: tx_test=(1500, 1024)
Data preprocessed and normalized (on CPU).
Train pairs: 112500, Validation pairs: 12500
Initializing FlowTranslatorV3 (flow_steps=4, latent_dim=1536, hidden=4096)

--- Starting V3 Flow+CUSA Translator (RUN V3_flow_cusa) ---
Training (Epochs: 60, LR=0.0003, WD=0.0004, L_NEG=1.0, L_CSA=0.1, L_USA=0.05, L_NDGL=0.01, L_HNR=0.003)...



Epoch 001/60:   0%|          | 0/220 [00:00<?, ?it/s]

CSLS (Pass 1/2):   0%|          | 0/49 [00:00<?, ?it/s]

CSLS (Pass 2/2):   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 001: Loss=1.2104 (Pos=0.5116, Neg=0.6966, CSA=0.0000, USA=0.0459, NDGL=0.0000, HNR=0.0000) (LR=3.0E-04, scale=1.071, NDGL_on=False) | R@10(ValSub)=0.2076 <- Best R@10! Saved.


...

Generating Sub:   0%|          | 0/2 [00:00<?, ?it/s]


✅ submission_V3_flow_cusa.csv saved successfully.
Calculating final validation scores on full val set...


CSLS (Pass 1/2):   0%|          | 0/49 [00:00<?, ?it/s]

CSLS (Pass 2/2):   0%|          | 0/13 [00:00<?, ?it/s]


--- Full Validation Results ---
Validation Recall@1:  0.1594  
Validation Recall@5:  0.3355  
Validation Recall@10: 0.4382  
Validation Recall@50: 0.6882  


In [32]:
import os
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm

# -----------------------------------------------------------------
# V4: Simple MLP Translator + SigLIP + L2 regression to image latent
#      - NO EMA
#      - NO NDGL / HNR
#      - NO USA / CSA
# -----------------------------------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------------------------------------------
# 1. Load data
# -----------------------------------------------------------------

train_data = np.load("/kaggle/input/aml-competition/train/train/train.npz")
test_data = np.load("/kaggle/input/aml-competition/test/test/test.clean.npz")

tx_train = train_data["captions/embeddings"]  # (N_caps, 1024)
im_train = train_data["images/embeddings"]    # (N_img, 1536)
tx_test = test_data["captions/embeddings"]    # (N_test, 1024)

# Assume 5 captions per image
repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)

print(f"Train shapes: tx={tx_train.shape}, im={im_train.shape}, im_expanded={im_train_expanded.shape}")
print(f"Test shape: tx_test={tx_test.shape}")

# -----------------------------------------------------------------
# 2. Preprocessing (CPU, normalization)
# -----------------------------------------------------------------
cpu_device = torch.device("cpu")

tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=cpu_device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=cpu_device)
tx_test_t = torch.as_tensor(tx_test, dtype=torch.float32, device=cpu_device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)

tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=cpu_device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)

print("Data preprocessed and normalized (on CPU).")

# -----------------------------------------------------------------
# 3. Translator Model (MLP)
# -----------------------------------------------------------------
class SimpleTranslatorV4(nn.Module):
    """
    Simple MLP translator:
      - 1024-d text -> 1536-d image latent
      - outputs raw latent (z_raw); normalized on demand
      - learnable logit_scale for similarity
    """
    def __init__(self, input_dim=1024, hidden_dim=8192, output_dim=1536, dropout=0.4):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, output_dim),
        )
        self.logit_scale = nn.Parameter(torch.tensor(0.0))  # exp(0)=1

        print(f"Initializing SimpleTranslatorV4 (hidden_dim={hidden_dim}, output_dim={output_dim})")

    def forward(self, x):
        # raw latent (non-normalizzato)
        return self.encoder(x)

    def encode_norm(self, x):
        z_raw = self.encoder(x)
        z = F.normalize(z_raw, p=2, dim=1)
        return z

    def get_logit_scale(self):
        return self.logit_scale.exp()


# -----------------------------------------------------------------
# 4. V4 Loss: SigLIP + L2 regression
# -----------------------------------------------------------------
LAMBDA_REG = 0.5        # weight for L2 regression to image latent
LOGIT_SCALE_MIN = -4.0  # clamp range
LOGIT_SCALE_MAX = 4.0

def siglip_loss_with_reg(
    z_raw,
    img_raw,
    img_indices,
    logit_scale,
    lambda_reg=LAMBDA_REG,
):
    """
    z_raw: (B, d) raw translator latent
    img_raw: (B, d) image latent (already normalized in our pipeline)
    img_indices: (B,) underlying image index for multi-positives
    logit_scale: scalar tensor exp(logit_scale_param)
    """
    device = z_raw.device
    B = z_raw.size(0)

    # normalize for contrastive part
    z = F.normalize(z_raw, p=2, dim=1)
    v = F.normalize(img_raw, p=2, dim=1)

    sims = logit_scale * (z @ v.T)  # (B, B)

    img_idx = img_indices.to(device)
    pos_mask = (img_idx.unsqueeze(1) == img_idx.unsqueeze(0))
    neg_mask = ~pos_mask

    if pos_mask.sum() == 0:
        pos_mask = torch.eye(B, dtype=torch.bool, device=device)
        neg_mask = ~pos_mask

    pos_scores = sims[pos_mask]
    neg_scores = sims[neg_mask]

    loss_pos = -F.logsigmoid(pos_scores).mean()
    loss_neg = -F.logsigmoid(-neg_scores).mean()
    loss_contrastive = loss_pos + loss_neg

    # L2 regression to image latent (raw)
    loss_reg = F.mse_loss(z_raw, img_raw)

    total_loss = loss_contrastive + lambda_reg * loss_reg

    return total_loss, {
        "loss_pos": loss_pos.detach(),
        "loss_neg": loss_neg.detach(),
        "loss_reg": loss_reg.detach(),
        "loss_contrastive": loss_contrastive.detach(),
    }

# -----------------------------------------------------------------
# 5. Validation split and DataLoader
# -----------------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)

idx_cpu = torch.randperm(N, device=cpu_device)
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

img_indices_full = torch.arange(N, device=cpu_device) // 5

img_indices_train = img_indices_full[train_idx]
img_indices_val = img_indices_full[val_idx]

tx_val_t = tx_train_t[val_idx]
tx_train_t_sub = tx_train_t[train_idx]
im_train_exp_sub = im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)

print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# -----------------------------------------------------------------
# 6. CSLS Recall@K for validation
# -----------------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, chunk_size=1024, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    device = next(model.parameters()).device
    all_correct_in_top_k = {k: [] for k in ks}

    Q_embed_list = []
    for i in range(0, len(tx_queries), chunk_size):
        chunk = tx_queries[i:i+chunk_size].to(device)
        Q_embed_list.append(model.encode_norm(chunk))
    Q_embed = torch.cat(Q_embed_list, dim=0)

    D_embed = im_database.to(device)

    # CSLS pass 1
    mean_knn_d_list = []
    for i in tqdm(range(0, len(D_embed), 512), desc="CSLS (Pass 1/2)", leave=False):
        sim_T_chunk = D_embed[i:i+512] @ Q_embed.T
        knn_d_chunk = torch.topk(sim_T_chunk, k=k_csls, dim=1).values
        mean_knn_d_list.append(knn_d_chunk.mean(1))
    mean_knn_d = torch.cat(mean_knn_d_list, dim=0).unsqueeze(0)

    gt = query_img_indices.to(device)

    # CSLS pass 2
    for i in tqdm(range(0, len(Q_embed), chunk_size), desc="CSLS (Pass 2/2)", leave=False):
        sim_chunk = Q_embed[i:i+chunk_size] @ D_embed.T
        gt_chunk = gt[i:i+chunk_size]

        knn_q_chunk = torch.topk(sim_chunk, k=k_csls, dim=1).values
        mean_knn_q = knn_q_chunk.mean(1, keepdim=True)

        csls_sim_chunk = 2 * sim_chunk - mean_knn_q - mean_knn_d
        top_indices_chunk = torch.argsort(csls_sim_chunk, dim=1, descending=True)

        for k in ks:
            top_k_preds = top_indices_chunk[:, :k]
            correct_in_top_k = (top_k_preds == gt_chunk.unsqueeze(1)).any(dim=1)
            all_correct_in_top_k[k].append(correct_in_top_k)

    recalls = {}
    for k in ks:
        recalls[f"R@{k}"] = torch.cat(all_correct_in_top_k[k]).float().mean().item()
    return recalls

# -----------------------------------------------------------------
# 7. Training loop
# -----------------------------------------------------------------
EPOCHS = 60
START_LR = 3e-4
WEIGHT_DECAY = 4e-4

RUN_ID = "V4_simple_siglip_l2"
FINAL_SAVE_PATH = f"{RUN_ID}_best.pth"

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

model = SimpleTranslatorV4(
    input_dim=1024,
    hidden_dim=8192,
    output_dim=1536,
    dropout=0.4,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=START_LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.amp.GradScaler()

print(f"\n--- Starting V4 Simple SigLIP+L2 Translator (RUN {RUN_ID}) ---")
print(
    f"Training (Epochs: {EPOCHS}, LR={START_LR}, WD={WEIGHT_DECAY}, "
    f"LAMBDA_REG={LAMBDA_REG})...\n"
)

best_val_r10 = 0.0

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0
    total_pos = 0.0
    total_neg = 0.0
    total_reg = 0.0
    total_contrastive = 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch:03d}/{EPOCHS}", leave=False):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        img_indices = img_indices.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type=device.type if device.type in ["cuda", "cpu"] else "cpu"):
            z_raw = model(x_batch)
            logit_scale = model.get_logit_scale()

            loss, loss_dict = siglip_loss_with_reg(
                z_raw=z_raw,
                img_raw=y_batch,
                img_indices=img_indices,
                logit_scale=logit_scale,
                lambda_reg=LAMBDA_REG,
            )

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        with torch.no_grad():
            model.logit_scale.clamp_(LOGIT_SCALE_MIN, LOGIT_SCALE_MAX)

        total_loss += loss.item()
        total_pos += loss_dict["loss_pos"].item()
        total_neg += loss_dict["loss_neg"].item()
        total_reg += loss_dict["loss_reg"].item()
        total_contrastive += loss_dict["loss_contrastive"].item()

    current_lr = scheduler.get_last_lr()[0]
    scheduler.step()

    num_batches = len(train_loader)
    avg_loss = total_loss / num_batches
    avg_pos = total_pos / num_batches
    avg_neg = total_neg / num_batches
    avg_reg = total_reg / num_batches
    avg_contrastive = total_contrastive / num_batches
    current_scale = model.get_logit_scale().item()

    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset, chunk_size=1024)
    current_r10 = rec['R@10']

    save_marker = ""
    if current_r10 > best_val_r10:
        best_val_r10 = current_r10
        torch.save(model.state_dict(), FINAL_SAVE_PATH)
        save_marker = "<- Best R@10! Saved."

    print(
        f"Epoch {epoch:03d}: "
        f"Loss={avg_loss:.4f} (Pos={avg_pos:.4f}, Neg={avg_neg:.4f}, Reg={avg_reg:.4f}, Ctr={avg_contrastive:.4f}) "
        f"(LR={current_lr:.1E}, scale={current_scale:.3f}) "
        f"| R@10(ValSub)={current_r10:.4f} {save_marker}"
    )

print(f"\n🎯 Training complete. Best model (R@10={best_val_r10:.4f}) saved as {FINAL_SAVE_PATH}")

# -----------------------------------------------------------------
# 8. Inference + Submission
# -----------------------------------------------------------------
print(f"Loading BEST model from {FINAL_SAVE_PATH} for inference...")
model = SimpleTranslatorV4(
    input_dim=1024,
    hidden_dim=8192,
    output_dim=1536,
    dropout=0.4,
).to(device)
model.load_state_dict(torch.load(FINAL_SAVE_PATH, map_location=device))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in tqdm(range(0, len(tx_test_t), 1024), desc="Generating Sub"):
        chunk = tx_test_t[i:i+1024].to(device)
        preds_list.append(model.encode_norm(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})

submission_filename = f"submission_{RUN_ID}.csv"
submission.to_csv(submission_filename, index=False)
print(f"\n✅ {submission_filename} saved successfully.")

# -----------------------------------------------------------------
# 9. Final Validation on Full Val Set
# -----------------------------------------------------------------
print("Calculating final validation scores on full val set...")
rec_full = recall_at_k(model, tx_val_t, im_train_t_unique, img_indices_val, chunk_size=1024)
print("\n--- Full Validation Results ---")
print(f"Validation Recall@1:  {rec_full['R@1']:<8.4f}")
print(f"Validation Recall@5:  {rec_full['R@5']:<8.4f}")
print(f"Validation Recall@10: {rec_full['R@10']:<8.4f}")
print(f"Validation Recall@50: {rec_full['R@50']:<8.4f}")


Using device: cuda
Train shapes: tx=(125000, 1024), im=(25000, 1536), im_expanded=(125000, 1536)
Test shape: tx_test=(1500, 1024)
Data preprocessed and normalized (on CPU).
Train pairs: 112500, Validation pairs: 12500
Initializing SimpleTranslatorV4 (hidden_dim=8192, output_dim=1536)

--- Starting V4 Simple SigLIP+L2 Translator (RUN V4_simple_siglip_l2) ---
Training (Epochs: 60, LR=0.0003, WD=0.0004, LAMBDA_REG=0.5)...



Epoch 001/60:   0%|          | 0/220 [00:00<?, ?it/s]

CSLS (Pass 1/2):   0%|          | 0/49 [00:00<?, ?it/s]

CSLS (Pass 2/2):   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 001: Loss=1.2855 (Pos=0.5503, Neg=0.6957, Reg=0.0790, Ctr=1.2460) (LR=3.0E-04, scale=1.072) | R@10(ValSub)=0.1536 <- Best R@10! Saved.


...

Generating Sub:   0%|          | 0/2 [00:00<?, ?it/s]


✅ submission_V4_simple_siglip_l2.csv saved successfully.
Calculating final validation scores on full val set...


CSLS (Pass 1/2):   0%|          | 0/49 [00:00<?, ?it/s]

CSLS (Pass 2/2):   0%|          | 0/13 [00:00<?, ?it/s]


--- Full Validation Results ---
Validation Recall@1:  0.1914  
Validation Recall@5:  0.3852  
Validation Recall@10: 0.4883  
Validation Recall@50: 0.7365  


In [3]:
import os
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm

# -----------------------------------------------------------------
# V5: Simple MLP Translator + SigLIP
#     + L2 to image latent (VAE-lite)
#     + optional distillation from V0 teacher text->image latent
#     + local soft-label intra-text clustering
#     + simple hard-negative margin
#     + temperature (logit_scale) regularization
# -----------------------------------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------------------------------------------
# 1. Load base data (same as before)
# -----------------------------------------------------------------
train_data = np.load("/kaggle/input/aml-competition/train/train/train.npz")
test_data = np.load("/kaggle/input/aml-competition/test/test/test.clean.npz")

tx_train = train_data["captions/embeddings"]  # (N_caps, 1024)
im_train = train_data["images/embeddings"]    # (N_img, 1536)
tx_test = test_data["captions/embeddings"]    # (N_test, 1024)

# Assume 5 captions per image
repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)

print(f"Train shapes: tx={tx_train.shape}, im={im_train.shape}, im_expanded={im_train_expanded.shape}")
print(f"Test shape: tx_test={tx_test.shape}")

# -----------------------------------------------------------------
# 1.b (optional) Teacher embeddings from V0 (distillation)
# -----------------------------------------------------------------
# IDEA: salvi offline da V0 un file tipo:
#   np.savez("v0_teacher_train_embeddings.npz", teacher_tx=teacher_tx_emb)
# dove teacher_tx_emb ha shape (N_caps, 1536) = output latente immagine predetto da V0 per ogni caption.
TEACHER_EMB_PATH = "v0_teacher_train_embeddings.npz"

USE_TEACHER = False
teacher_tx_t = None

if os.path.exists(TEACHER_EMB_PATH):
    print(f"Found teacher embeddings at '{TEACHER_EMB_PATH}'. Loading for distillation...")
    teacher_data = np.load(TEACHER_EMB_PATH)
    if "teacher_tx" not in teacher_data:
        print("WARNING: 'teacher_tx' key not found in teacher npz, distillation disabled.")
    else:
        teacher_tx = teacher_data["teacher_tx"]
        if teacher_tx.shape[0] != tx_train.shape[0]:
            print(
                f"WARNING: teacher_tx shape {teacher_tx.shape} does not match tx_train {tx_train.shape}. "
                "Distillation disabled."
            )
        else:
            teacher_tx_t = torch.as_tensor(teacher_tx, dtype=torch.float32)
            USE_TEACHER = True
            print("Teacher distillation ENABLED.")
else:
    print(f"No teacher embeddings found at '{TEACHER_EMB_PATH}'. Distillation OFF.")

# -----------------------------------------------------------------
# 2. Preprocessing (CPU, normalization)
# -----------------------------------------------------------------
cpu_device = torch.device("cpu")

tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=cpu_device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=cpu_device)
tx_test_t = torch.as_tensor(tx_test, dtype=torch.float32, device=cpu_device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)

tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=cpu_device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)

if USE_TEACHER:
    # portiamo teacher_tx_t su CPU, poi (se serve) su device in batch
    teacher_tx_t = teacher_tx_t.to(cpu_device)

print("Data preprocessed and normalized (on CPU).")

# -----------------------------------------------------------------
# 3. Translator Model (MLP)
# -----------------------------------------------------------------
class TranslatorV5(nn.Module):
    """
    Simple MLP translator:
      - 1024-d text -> 1536-d image latent
      - outputs raw latent (z_raw); normalized on demand
      - learnable logit_scale for similarity
    """
    def __init__(self, input_dim=1024, hidden_dim=8192, output_dim=1536, dropout=0.4):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, output_dim),
        )
        # start with moderate scale, e.g. log(10)
        self.logit_scale = nn.Parameter(torch.tensor(2.3))  # exp(2.3) ~ 10

        print(f"Initializing TranslatorV5 (hidden_dim={hidden_dim}, output_dim={output_dim})")

    def forward(self, x):
        # raw latent (non-normalizzato)
        return self.encoder(x)

    def encode_norm(self, x):
        z_raw = self.encoder(x)
        z = F.normalize(z_raw, p=2, dim=1)
        return z

    def get_logit_scale(self):
        return self.logit_scale.exp()

# -----------------------------------------------------------------
# 4. V5 Loss: SigLIP + L2 + optional teacher + local soft + margin + temp reg
# -----------------------------------------------------------------
LAMBDA_REG_IMG     = 0.5    # weight for L2 regression to image latent
LAMBDA_REG_TEACHER = 0.5    # weight for L2 distillation to teacher latent
LAMBDA_SOFT_LOCAL  = 0.02   # weight for local soft-label intra-text clustering
LAMBDA_MARGIN      = 0.1    # weight for hard-negative margin loss
LAMBDA_TEMP_REG    = 1e-3   # weight for logit_scale^2 regularization

LOCAL_SIM_THRESHOLD = 0.7   # threshold on text-text sim to be considered "near" (for soft local clustering)
MARGIN_M            = 0.05  # margin for hardest negative hinge

LOGIT_SCALE_MIN = -1.5      # clamp range (scale ~ 0.22)
LOGIT_SCALE_MAX = 3.0       # clamp range (scale ~ 20)

def v5_loss(
    x_batch,       # (B, 1024) normalized text embeddings (base)
    z_raw,         # (B, 1536) raw translator outputs
    img_raw,       # (B, 1536) expanded image latents (normalized)
    img_indices,   # (B,) img index for multi-positives
    logit_scale,   # scalar exp(logit_scale_param)
    teacher_batch=None,  # (B, 1536) teacher latents (optional)
):
    device = z_raw.device
    B = z_raw.size(0)

    # Normalize for contrastive part
    z = F.normalize(z_raw, p=2, dim=1)
    v = F.normalize(img_raw, p=2, dim=1)

    # ------------------------------------------------------------
    # 4.1 Core SigLIP loss (multi-positives)
    # ------------------------------------------------------------
    sims = logit_scale * (z @ v.T)  # (B, B)

    img_idx = img_indices.to(device)
    pos_mask = (img_idx.unsqueeze(1) == img_idx.unsqueeze(0))
    neg_mask = ~pos_mask

    if pos_mask.sum() == 0:
        pos_mask = torch.eye(B, dtype=torch.bool, device=device)
        neg_mask = ~pos_mask

    pos_scores = sims[pos_mask]
    neg_scores = sims[neg_mask]

    loss_pos = -F.logsigmoid(pos_scores).mean()
    loss_neg = -F.logsigmoid(-neg_scores).mean()
    loss_contrastive = loss_pos + loss_neg

    # ------------------------------------------------------------
    # 4.2 L2 regression to image latent (VAE-lite)
    # ------------------------------------------------------------
    loss_reg_img = F.mse_loss(z_raw, img_raw)

    # ------------------------------------------------------------
    # 4.3 Optional distillation to teacher latent (if provided)
    # ------------------------------------------------------------
    if teacher_batch is not None:
        # teacher_batch assumed 1536-d raw latent
        loss_reg_teacher = F.mse_loss(z_raw, teacher_batch.to(device))
    else:
        loss_reg_teacher = torch.tensor(0.0, device=device)

    # ------------------------------------------------------------
    # 4.4 Local soft-label intra-text clustering (USA-local)
    #     - use x_batch (1024-d normalized text features) to find near neighbors
    # ------------------------------------------------------------
    # x_batch is already normalized (tx_train_t was normalized earlier)
    with torch.no_grad():
        sim_tt = x_batch @ x_batch.T  # (B, B)
        # mask of near neighbors: sim > threshold and not self
        near_mask = (sim_tt > LOCAL_SIM_THRESHOLD).fill_diagonal_(False)

    if near_mask.any():
        # for each pair (i,j) where near_mask[i,j]=True, minimize ||z_raw[i] - z_raw[j]||^2
        diff = z_raw.unsqueeze(1) - z_raw.unsqueeze(0)  # (B, B, d)
        dist2 = (diff ** 2).sum(dim=-1)  # (B, B)
        loss_soft_local = (dist2 * near_mask).sum() / near_mask.sum()
    else:
        loss_soft_local = torch.tensor(0.0, device=device)

    # ------------------------------------------------------------
    # 4.5 Hard-negative margin (per query)
    #     - for ogni i, confronto s_pos vs s_neg_hard
    # ------------------------------------------------------------
    # positive score per i: massimo sim con i positivi
    sims_pos_only = sims.masked_fill(~pos_mask, float("-inf"))
    pos_best, _ = sims_pos_only.max(dim=1)  # (B,)

    sims_neg_only = sims.masked_fill(pos_mask, float("-inf"))
    neg_best, _ = sims_neg_only.max(dim=1)  # (B,)

    # margin hinge: max(0, m + s_neg_hard - s_pos)
    margin_gap = MARGIN_M + neg_best - pos_best
    loss_margin = F.relu(margin_gap).mean()

    # ------------------------------------------------------------
    # 4.6 Temperature regularization (logit_scale parameter)
    # ------------------------------------------------------------
    # logit_scale is exp(param); we penalize the parameter (log-scale)
    # to keep scale in a moderate range.
    loss_temp = logit_scale.log().pow(2)  # scalar

    # ------------------------------------------------------------
    # 4.7 Combine
    # ------------------------------------------------------------
    total_loss = (
        loss_contrastive
        + LAMBDA_REG_IMG * loss_reg_img
        + LAMBDA_REG_TEACHER * loss_reg_teacher
        + LAMBDA_SOFT_LOCAL * loss_soft_local
        + LAMBDA_MARGIN * loss_margin
        + LAMBDA_TEMP_REG * loss_temp
    )

    loss_dict = {
        "loss_pos": loss_pos.detach(),
        "loss_neg": loss_neg.detach(),
        "loss_contrastive": loss_contrastive.detach(),
        "loss_reg_img": loss_reg_img.detach(),
        "loss_reg_teacher": loss_reg_teacher.detach(),
        "loss_soft_local": loss_soft_local.detach(),
        "loss_margin": loss_margin.detach(),
        "loss_temp": loss_temp.detach(),
    }

    return total_loss, loss_dict

# -----------------------------------------------------------------
# 5. Validation split and DataLoader
# -----------------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)

idx_cpu = torch.randperm(N, device=cpu_device)
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

img_indices_full = torch.arange(N, device=cpu_device) // 5

img_indices_train = img_indices_full[train_idx]
img_indices_val = img_indices_full[val_idx]

tx_val_t = tx_train_t[val_idx]
tx_train_t_sub = tx_train_t[train_idx]
im_train_exp_sub = im_train_exp[train_idx]

if USE_TEACHER:
    teacher_tx_sub = teacher_tx_t[train_idx]
else:
    teacher_tx_sub = None

if USE_TEACHER:
    train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train, teacher_tx_sub)
else:
    train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)

print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# -----------------------------------------------------------------
# 6. CSLS Recall@K for validation (same as before)
# -----------------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, chunk_size=1024, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    device = next(model.parameters()).device
    all_correct_in_top_k = {k: [] for k in ks}

    Q_embed_list = []
    for i in range(0, len(tx_queries), chunk_size):
        chunk = tx_queries[i:i+chunk_size].to(device)
        Q_embed_list.append(model.encode_norm(chunk))
    Q_embed = torch.cat(Q_embed_list, dim=0)

    D_embed = im_database.to(device)

    # CSLS pass 1
    mean_knn_d_list = []
    for i in tqdm(range(0, len(D_embed), 512), desc="CSLS (Pass 1/2)", leave=False):
        sim_T_chunk = D_embed[i:i+512] @ Q_embed.T
        knn_d_chunk = torch.topk(sim_T_chunk, k=k_csls, dim=1).values
        mean_knn_d_list.append(knn_d_chunk.mean(1))
    mean_knn_d = torch.cat(mean_knn_d_list, dim=0).unsqueeze(0)

    gt = query_img_indices.to(device)

    # CSLS pass 2
    for i in tqdm(range(0, len(Q_embed), chunk_size), desc="CSLS (Pass 2/2)", leave=False):
        sim_chunk = Q_embed[i:i+chunk_size] @ D_embed.T
        gt_chunk = gt[i:i+chunk_size]

        knn_q_chunk = torch.topk(sim_chunk, k=k_csls, dim=1).values
        mean_knn_q = knn_q_chunk.mean(1, keepdim=True)

        csls_sim_chunk = 2 * sim_chunk - mean_knn_q - mean_knn_d
        top_indices_chunk = torch.argsort(csls_sim_chunk, dim=1, descending=True)

        for k in ks:
            top_k_preds = top_indices_chunk[:, :k]
            correct_in_top_k = (top_k_preds == gt_chunk.unsqueeze(1)).any(dim=1)
            all_correct_in_top_k[k].append(correct_in_top_k)

    recalls = {}
    for k in ks:
        recalls[f"R@{k}"] = torch.cat(all_correct_in_top_k[k]).float().mean().item()
    return recalls

# -----------------------------------------------------------------
# 7. Training loop
# -----------------------------------------------------------------
EPOCHS = 60
START_LR = 3e-4
WEIGHT_DECAY = 4e-4

RUN_ID = "V5_distill_local"
FINAL_SAVE_PATH = f"{RUN_ID}_best.pth"

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

model = TranslatorV5(
    input_dim=1024,
    hidden_dim=8192,
    output_dim=1536,
    dropout=0.4,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=START_LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.amp.GradScaler()

print(f"\n--- Starting V5 Distill+Local Translator (RUN {RUN_ID}) ---")
print(
    f"Training (Epochs: {EPOCHS}, LR={START_LR}, WD={WEIGHT_DECAY}, "
    f"L_REG_IMG={LAMBDA_REG_IMG}, L_REG_TEACHER={LAMBDA_REG_TEACHER}, "
    f"L_SOFT_LOCAL={LAMBDA_SOFT_LOCAL}, L_MARGIN={LAMBDA_MARGIN}, "
    f"L_TEMP_REG={LAMBDA_TEMP_REG})...\n"
)

best_val_r10 = 0.0

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0
    total_pos = 0.0
    total_neg = 0.0
    total_reg_img = 0.0
    total_reg_teacher = 0.0
    total_soft_local = 0.0
    total_margin = 0.0
    total_temp = 0.0
    total_contrastive = 0.0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch:03d}/{EPOCHS}", leave=False):
        if USE_TEACHER:
            x_batch, y_batch, img_indices, teacher_batch = batch
            teacher_batch = teacher_batch.to(device)
        else:
            x_batch, y_batch, img_indices = batch
            teacher_batch = None

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        img_indices = img_indices.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type=device.type if device.type in ["cuda", "cpu"] else "cpu"):
            z_raw = model(x_batch)
            logit_scale = model.get_logit_scale()

            loss, loss_dict = v5_loss(
                x_batch=x_batch,
                z_raw=z_raw,
                img_raw=y_batch,
                img_indices=img_indices,
                logit_scale=logit_scale,
                teacher_batch=teacher_batch,
            )

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        with torch.no_grad():
            # Clamp the underlying parameter (log-scale) into [LOGIT_SCALE_MIN, LOGIT_SCALE_MAX]
            model.logit_scale.clamp_(LOGIT_SCALE_MIN, LOGIT_SCALE_MAX)

        total_loss += loss.item()
        total_pos += loss_dict["loss_pos"].item()
        total_neg += loss_dict["loss_neg"].item()
        total_reg_img += loss_dict["loss_reg_img"].item()
        total_reg_teacher += loss_dict["loss_reg_teacher"].item()
        total_soft_local += loss_dict["loss_soft_local"].item()
        total_margin += loss_dict["loss_margin"].item()
        total_temp += loss_dict["loss_temp"].item()
        total_contrastive += loss_dict["loss_contrastive"].item()

    current_lr = scheduler.get_last_lr()[0]
    scheduler.step()

    num_batches = len(train_loader)
    avg_loss = total_loss / num_batches
    avg_pos = total_pos / num_batches
    avg_neg = total_neg / num_batches
    avg_reg_img = total_reg_img / num_batches
    avg_reg_teacher = total_reg_teacher / num_batches
    avg_soft_local = total_soft_local / num_batches
    avg_margin = total_margin / num_batches
    avg_temp = total_temp / num_batches
    avg_contrastive = total_contrastive / num_batches
    current_scale = model.get_logit_scale().item()

    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset, chunk_size=1024)
    current_r10 = rec['R@10']

    save_marker = ""
    if current_r10 > best_val_r10:
        best_val_r10 = current_r10
        torch.save(model.state_dict(), FINAL_SAVE_PATH)
        save_marker = "<- Best R@10! Saved."

    print(
        f"Epoch {epoch:03d}: "
        f"Loss={avg_loss:.4f} "
        f"(Pos={avg_pos:.4f}, Neg={avg_neg:.4f}, Ctr={avg_contrastive:.4f}, "
        f"RegImg={avg_reg_img:.4f}, RegTeach={avg_reg_teacher:.4f}, "
        f"SoftLoc={avg_soft_local:.4f}, Margin={avg_margin:.4f}, Temp={avg_temp:.4f}) "
        f"(LR={current_lr:.1E}, scale={current_scale:.3f}) "
        f"| R@10(ValSub)={current_r10:.4f} {save_marker}"
    )

print(f"\n🎯 Training complete. Best model (R@10={best_val_r10:.4f}) saved as {FINAL_SAVE_PATH}")

# -----------------------------------------------------------------
# 8. Inference + Submission
# -----------------------------------------------------------------
print(f"Loading BEST model from {FINAL_SAVE_PATH} for inference...")
model = TranslatorV5(
    input_dim=1024,
    hidden_dim=8192,
    output_dim=1536,
    dropout=0.4,
).to(device)
model.load_state_dict(torch.load(FINAL_SAVE_PATH, map_location=device))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in tqdm(range(0, len(tx_test_t), 1024), desc="Generating Sub"):
        chunk = tx_test_t[i:i+1024].to(device)
        preds_list.append(model.encode_norm(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})

submission_filename = f"submission_{RUN_ID}.csv"
submission.to_csv(submission_filename, index=False)
print(f"\n✅ {submission_filename} saved successfully.")

# -----------------------------------------------------------------
# 9. Final Validation on Full Val Set
# -----------------------------------------------------------------
print("Calculating final validation scores on full val set...")
rec_full = recall_at_k(model, tx_val_t, im_train_t_unique, img_indices_val, chunk_size=1024)
print("\n--- Full Validation Results ---")
print(f"Validation Recall@1:  {rec_full['R@1']:<8.4f}")
print(f"Validation Recall@5:  {rec_full['R@5']:<8.4f}")
print(f"Validation Recall@10: {rec_full['R@10']:<8.4f}")
print(f"Validation Recall@50: {rec_full['R@50']:<8.4f}")

Using device: cuda
Train shapes: tx=(125000, 1024), im=(25000, 1536), im_expanded=(125000, 1536)
Test shape: tx_test=(1500, 1024)
No teacher embeddings found at 'v0_teacher_train_embeddings.npz'. Distillation OFF.
Data preprocessed and normalized (on CPU).
Train pairs: 112500, Validation pairs: 12500
Initializing TranslatorV5 (hidden_dim=8192, output_dim=1536)

--- Starting V5 Distill+Local Translator (RUN V5_distill_local) ---
Training (Epochs: 60, LR=0.0003, WD=0.0004, L_REG_IMG=0.5, L_REG_TEACHER=0.5, L_SOFT_LOCAL=0.02, L_MARGIN=0.1, L_TEMP_REG=0.001)...



Epoch 001/60:   0%|          | 0/220 [00:00<?, ?it/s]

CSLS (Pass 1/2):   0%|          | 0/49 [00:00<?, ?it/s]

CSLS (Pass 2/2):   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 001: Loss=5.0436 (Pos=0.4197, Neg=0.7348, Ctr=1.1545, RegImg=0.0854, RegTeach=0.0000, SoftLoc=188.6675, Margin=0.6772, Temp=5.3325) (LR=3.0E-04, scale=10.375) | R@10(ValSub)=0.0730 <- Best R@10! Saved.


...

Generating Sub:   0%|          | 0/2 [00:00<?, ?it/s]


✅ submission_V5_distill_local.csv saved successfully.
Calculating final validation scores on full val set...


CSLS (Pass 1/2):   0%|          | 0/49 [00:00<?, ?it/s]

CSLS (Pass 2/2):   0%|          | 0/13 [00:00<?, ?it/s]


--- Full Validation Results ---
Validation Recall@1:  0.2722  
Validation Recall@5:  0.5157  
Validation Recall@10: 0.6250  
Validation Recall@50: 0.8322  


In [2]:
import os
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm

# -----------------------------------------------------------------
# V6: Simple MLP Translator + SigLIP
#     + L2 to image latent
#     + local soft-label (text-text)
#     + same-image caption clustering
#     + top-k hard-negative margin
#     + small VAE-like prior on z_raw
#     + fixed temperature (no logit_scale param)
# -----------------------------------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------------------------------------------
# 1. Load base data
# -----------------------------------------------------------------
train_data = np.load("/kaggle/input/aml-competition/train/train/train.npz")
test_data = np.load("/kaggle/input/aml-competition/test/test/test.clean.npz")

tx_train = train_data["captions/embeddings"]  # (N_caps, 1024)
im_train = train_data["images/embeddings"]    # (N_img, 1536)
tx_test = test_data["captions/embeddings"]    # (N_test, 1024)

# Assume 5 captions per image
repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)

print(f"Train shapes: tx={tx_train.shape}, im={im_train.shape}, im_expanded={im_train_expanded.shape}")
print(f"Test shape: tx_test={tx_test.shape}")

# -----------------------------------------------------------------
# 2. Preprocessing (CPU, normalization)
# -----------------------------------------------------------------
cpu_device = torch.device("cpu")

tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=cpu_device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=cpu_device)
tx_test_t = torch.as_tensor(tx_test, dtype=torch.float32, device=cpu_device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)

tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=cpu_device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)

print("Data preprocessed and normalized (on CPU).")

# -----------------------------------------------------------------
# 3. Translator Model (MLP)
# -----------------------------------------------------------------
class TranslatorV6(nn.Module):
    """
    Simple MLP translator:
      - 1024-d text -> 1536-d image latent
      - outputs raw latent (z_raw); normalized on demand
      - fixed temperature for similarity
    """
    def __init__(self, input_dim=1024, hidden_dim=8192, output_dim=1536, dropout=0.4):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, output_dim),
        )
        # Fixed logit scale (no learnable parameter)
        self._fixed_logit_scale = 10.0

        print(f"Initializing TranslatorV6 (hidden_dim={hidden_dim}, output_dim={output_dim})")

    def forward(self, x):
        # raw latent (non-normalizzato)
        return self.encoder(x)

    def encode_norm(self, x):
        z_raw = self.encoder(x)
        z = F.normalize(z_raw, p=2, dim=1)
        return z

    def get_logit_scale(self):
        # fixed temperature
        return self._fixed_logit_scale

# -----------------------------------------------------------------
# 4. V6 Loss
# -----------------------------------------------------------------
LAMBDA_REG_IMG      = 0.5    # L2 to image latent
LAMBDA_SOFT_LOCAL   = 0.02   # local text-text soft-label
LAMBDA_SAME_CAPTION = 0.10   # cluster captions of same image
LAMBDA_MARGIN       = 0.05   # top-k hardest negative margin
LAMBDA_PRIOR        = 1e-3   # VAE-like prior on z_raw

LOCAL_SIM_THRESHOLD = 0.7    # threshold for text-text local neighborhood
MARGIN_M            = 0.05   # margin for hardest negatives
HNM_K               = 2      # top-k negatives per query for margin

def v6_loss(
    x_batch,       # (B, 1024) normalized text embeddings (base)
    z_raw,         # (B, 1536) raw translator outputs
    img_raw,       # (B, 1536) expanded image latents (normalized)
    img_indices,   # (B,) img index for multi-positives
):
    device = z_raw.device
    B = z_raw.size(0)

    # Normalize for contrastive part
    z = F.normalize(z_raw, p=2, dim=1)
    v = F.normalize(img_raw, p=2, dim=1)

    # ------------------------------------------------------------
    # 4.1 Core SigLIP loss (multi-positives)
    # ------------------------------------------------------------
    logit_scale = torch.tensor(10.0, dtype=z.dtype, device=device)
    sims = logit_scale * (z @ v.T)  # (B, B)

    img_idx = img_indices.to(device)
    pos_mask = (img_idx.unsqueeze(1) == img_idx.unsqueeze(0))
    neg_mask = ~pos_mask

    if pos_mask.sum() == 0:
        pos_mask = torch.eye(B, dtype=torch.bool, device=device)
        neg_mask = ~pos_mask

    pos_scores = sims[pos_mask]
    neg_scores = sims[neg_mask]

    loss_pos = -F.logsigmoid(pos_scores).mean()
    loss_neg = -F.logsigmoid(-neg_scores).mean()
    loss_contrastive = loss_pos + loss_neg

    # ------------------------------------------------------------
    # 4.2 L2 regression to image latent (VAE-lite)
    # ------------------------------------------------------------
    loss_reg_img = F.mse_loss(z_raw, img_raw)

    # ------------------------------------------------------------
    # 4.3 Caption-consistency: same image -> cluster z_raw
    # ------------------------------------------------------------
    same_mask = (img_idx.unsqueeze(1) == img_idx.unsqueeze(0))
    same_mask.fill_diagonal_(False)
    if same_mask.any():
        diff_same = z_raw.unsqueeze(1) - z_raw.unsqueeze(0)  # (B, B, d)
        dist2_same = (diff_same ** 2).sum(dim=-1)           # (B, B)
        loss_same_caption = (dist2_same * same_mask).sum() / same_mask.sum()
    else:
        loss_same_caption = torch.tensor(0.0, device=device)

    # ------------------------------------------------------------
    # 4.4 Local soft-label intra-text (USA-local, come V5)
    # ------------------------------------------------------------
    with torch.no_grad():
        sim_tt = x_batch @ x_batch.T  # (B, B), x_batch è già normalizzato
        near_mask = (sim_tt > LOCAL_SIM_THRESHOLD).fill_diagonal_(False)

    if near_mask.any():
        diff_local = z_raw.unsqueeze(1) - z_raw.unsqueeze(0)  # (B, B, d)
        dist2_local = (diff_local ** 2).sum(dim=-1)           # (B, B)
        loss_soft_local = (dist2_local * near_mask).sum() / near_mask.sum()
    else:
        loss_soft_local = torch.tensor(0.0, device=device)

    # ------------------------------------------------------------
    # 4.5 Top-k hard-negative margin (HNM-lite)
    # ------------------------------------------------------------
    sims_pos_only = sims.masked_fill(~pos_mask, float("-inf"))
    pos_best, _ = sims_pos_only.max(dim=1)  # (B,)

    sims_neg_only = sims.masked_fill(pos_mask, float("-inf"))
    # top-k negs
    k = min(HNM_K, max(1, B - 1))
    neg_topk = torch.topk(sims_neg_only, k=k, dim=1).values  # (B, k)

    # margin hinge: max(0, m + s_neg_topk - s_pos_best)
    margin_gaps = MARGIN_M + neg_topk - pos_best.unsqueeze(1)
    loss_margin = F.relu(margin_gaps).mean()

    # ------------------------------------------------------------
    # 4.6 VAE-like prior on z_raw: mean ~ 0, var ~ 1
    # ------------------------------------------------------------
    # NB: z_raw è nello spazio 1536, non normalizzato. Tenere peso piccolo.
    mean_z = z_raw.mean(dim=0)           # (d,)
    var_z = z_raw.var(dim=0, unbiased=False)  # (d,)
    loss_prior = mean_z.pow(2).mean() + (var_z - 1.0).pow(2).mean()

    # ------------------------------------------------------------
    # 4.7 Combine
    # ------------------------------------------------------------
    total_loss = (
        loss_contrastive
        + LAMBDA_REG_IMG * loss_reg_img
        + LAMBDA_SAME_CAPTION * loss_same_caption
        + LAMBDA_SOFT_LOCAL * loss_soft_local
        + LAMBDA_MARGIN * loss_margin
        + LAMBDA_PRIOR * loss_prior
    )

    loss_dict = {
        "loss_pos": loss_pos.detach(),
        "loss_neg": loss_neg.detach(),
        "loss_contrastive": loss_contrastive.detach(),
        "loss_reg_img": loss_reg_img.detach(),
        "loss_same_caption": loss_same_caption.detach(),
        "loss_soft_local": loss_soft_local.detach(),
        "loss_margin": loss_margin.detach(),
        "loss_prior": loss_prior.detach(),
    }

    return total_loss, loss_dict

# -----------------------------------------------------------------
# 5. Validation split and DataLoader
# -----------------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)

idx_cpu = torch.randperm(N, device=cpu_device)
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

img_indices_full = torch.arange(N, device=cpu_device) // 5

img_indices_train = img_indices_full[train_idx]
img_indices_val = img_indices_full[val_idx]

tx_val_t = tx_train_t[val_idx]
tx_train_t_sub = tx_train_t[train_idx]
im_train_exp_sub = im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)

print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# -----------------------------------------------------------------
# 6. CSLS Recall@K for validation
# -----------------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, chunk_size=1024, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    device = next(model.parameters()).device
    all_correct_in_top_k = {k: [] for k in ks}

    Q_embed_list = []
    for i in range(0, len(tx_queries), chunk_size):
        chunk = tx_queries[i:i+chunk_size].to(device)
        Q_embed_list.append(model.encode_norm(chunk))
    Q_embed = torch.cat(Q_embed_list, dim=0)

    D_embed = im_database.to(device)

    # CSLS pass 1
    mean_knn_d_list = []
    for i in tqdm(range(0, len(D_embed), 512), desc="CSLS (Pass 1/2)", leave=False):
        sim_T_chunk = D_embed[i:i+512] @ Q_embed.T
        knn_d_chunk = torch.topk(sim_T_chunk, k=k_csls, dim=1).values
        mean_knn_d_list.append(knn_d_chunk.mean(1))
    mean_knn_d = torch.cat(mean_knn_d_list, dim=0).unsqueeze(0)

    gt = query_img_indices.to(device)

    # CSLS pass 2
    for i in tqdm(range(0, len(Q_embed), chunk_size), desc="CSLS (Pass 2/2)", leave=False):
        sim_chunk = Q_embed[i:i+chunk_size] @ D_embed.T
        gt_chunk = gt[i:i+chunk_size]

        knn_q_chunk = torch.topk(sim_chunk, k=k_csls, dim=1).values
        mean_knn_q = knn_q_chunk.mean(1, keepdim=True)

        csls_sim_chunk = 2 * sim_chunk - mean_knn_q - mean_knn_d
        top_indices_chunk = torch.argsort(csls_sim_chunk, dim=1, descending=True)

        for k in ks:
            top_k_preds = top_indices_chunk[:, :k]
            correct_in_top_k = (top_k_preds == gt_chunk.unsqueeze(1)).any(dim=1)
            all_correct_in_top_k[k].append(correct_in_top_k)

    recalls = {}
    for k in ks:
        recalls[f"R@{k}"] = torch.cat(all_correct_in_top_k[k]).float().mean().item()
    return recalls

# -----------------------------------------------------------------
# 7. Training loop
# -----------------------------------------------------------------
EPOCHS = 60
START_LR = 3e-4
WEIGHT_DECAY = 4e-4

RUN_ID = "V6_multiobj"
FINAL_SAVE_PATH = f"{RUN_ID}_best.pth"

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

model = TranslatorV6(
    input_dim=1024,
    hidden_dim=8192,
    output_dim=1536,
    dropout=0.4,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=START_LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.amp.GradScaler()

print(f"\n--- Starting V6 Multi-Objective Translator (RUN {RUN_ID}) ---")
print(
    f"Training (Epochs: {EPOCHS}, LR={START_LR}, WD={WEIGHT_DECAY}, "
    f"L_REG_IMG={LAMBDA_REG_IMG}, L_SOFT_LOCAL={LAMBDA_SOFT_LOCAL}, "
    f"L_SAME_CAPTION={LAMBDA_SAME_CAPTION}, L_MARGIN={LAMBDA_MARGIN}, "
    f"L_PRIOR={LAMBDA_PRIOR}, HNM_K={HNM_K})...\n"
)

best_val_r10 = 0.0

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0
    total_pos = 0.0
    total_neg = 0.0
    total_reg_img = 0.0
    total_soft_local = 0.0
    total_same_caption = 0.0
    total_margin = 0.0
    total_prior = 0.0
    total_contrastive = 0.0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch:03d}/{EPOCHS}", leave=False):
        x_batch, y_batch, img_indices = batch

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        img_indices = img_indices.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type=device.type if device.type in ["cuda", "cpu"] else "cpu"):
            z_raw = model(x_batch)

            loss, loss_dict = v6_loss(
                x_batch=x_batch,
                z_raw=z_raw,
                img_raw=y_batch,
                img_indices=img_indices,
            )

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        total_pos += loss_dict["loss_pos"].item()
        total_neg += loss_dict["loss_neg"].item()
        total_reg_img += loss_dict["loss_reg_img"].item()
        total_soft_local += loss_dict["loss_soft_local"].item()
        total_same_caption += loss_dict["loss_same_caption"].item()
        total_margin += loss_dict["loss_margin"].item()
        total_prior += loss_dict["loss_prior"].item()
        total_contrastive += loss_dict["loss_contrastive"].item()

    current_lr = scheduler.get_last_lr()[0]
    scheduler.step()

    num_batches = len(train_loader)
    avg_loss = total_loss / num_batches
    avg_pos = total_pos / num_batches
    avg_neg = total_neg / num_batches
    avg_reg_img = total_reg_img / num_batches
    avg_soft_local = total_soft_local / num_batches
    avg_same_caption = total_same_caption / num_batches
    avg_margin = total_margin / num_batches
    avg_prior = total_prior / num_batches
    avg_contrastive = total_contrastive / num_batches
    current_scale = model.get_logit_scale()

    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset, chunk_size=1024)
    current_r10 = rec['R@10']

    save_marker = ""
    if current_r10 > best_val_r10:
        best_val_r10 = current_r10
        torch.save(model.state_dict(), FINAL_SAVE_PATH)
        save_marker = "<- Best R@10! Saved."

    print(
        f"Epoch {epoch:03d}: "
        f"Loss={avg_loss:.4f} "
        f"(Pos={avg_pos:.4f}, Neg={avg_neg:.4f}, Ctr={avg_contrastive:.4f}, "
        f"RegImg={avg_reg_img:.4f}, SoftLoc={avg_soft_local:.4f}, "
        f"SameCap={avg_same_caption:.4f}, Margin={avg_margin:.4f}, "
        f"Prior={avg_prior:.4f}) "
        f"(LR={current_lr:.1E}, scale={current_scale:.3f}) "
        f"| R@10(ValSub)={current_r10:.4f} {save_marker}"
    )

print(f"\n🎯 Training complete. Best model (R@10={best_val_r10:.4f}) saved as {FINAL_SAVE_PATH}")

# -----------------------------------------------------------------
# 8. Inference + Submission
# -----------------------------------------------------------------
print(f"Loading BEST model from {FINAL_SAVE_PATH} for inference...")
model = TranslatorV6(
    input_dim=1024,
    hidden_dim=8192,
    output_dim=1536,
    dropout=0.4,
).to(device)
model.load_state_dict(torch.load(FINAL_SAVE_PATH, map_location=device))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in tqdm(range(0, len(tx_test_t), 1024), desc="Generating Sub"):
        chunk = tx_test_t[i:i+1024].to(device)
        preds_list.append(model.encode_norm(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})

submission_filename = f"submission_{RUN_ID}.csv"
submission.to_csv(submission_filename, index=False)
print(f"\n✅ {submission_filename} saved successfully.")

# -----------------------------------------------------------------
# 9. Final Validation on Full Val Set
# -----------------------------------------------------------------
print("Calculating final validation scores on full val set...")
rec_full = recall_at_k(model, tx_val_t, im_train_t_unique, img_indices_val, chunk_size=1024)
print("\n--- Full Validation Results ---")
print(f"Validation Recall@1:  {rec_full['R@1']:<8.4f}")
print(f"Validation Recall@5:  {rec_full['R@5']:<8.4f}")
print(f"Validation Recall@10: {rec_full['R@10']:<8.4f}")
print(f"Validation Recall@50: {rec_full['R@50']:<8.4f}")


Using device: cuda
Train shapes: tx=(125000, 1024), im=(25000, 1536), im_expanded=(125000, 1536)
Test shape: tx_test=(1500, 1024)
Data preprocessed and normalized (on CPU).
Train pairs: 112500, Validation pairs: 12500
Initializing TranslatorV6 (hidden_dim=8192, output_dim=1536)

--- Starting V6 Multi-Objective Translator (RUN V6_multiobj) ---
Training (Epochs: 60, LR=0.0003, WD=0.0004, L_REG_IMG=0.5, L_SOFT_LOCAL=0.02, L_SAME_CAPTION=0.1, L_MARGIN=0.05, L_PRIOR=0.001, HNM_K=2)...



Epoch 001/60:   0%|          | 0/220 [00:00<?, ?it/s]

CSLS (Pass 1/2):   0%|          | 0/49 [00:00<?, ?it/s]

CSLS (Pass 2/2):   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 001: Loss=39.4797 (Pos=0.6817, Neg=0.7001, Ctr=1.3818, RegImg=0.1808, SoftLoc=313.6198, SameCap=317.0141, Margin=0.6569, Prior=0.8827) (LR=3.0E-04, scale=10.000) | R@10(ValSub)=0.0060 <- Best R@10! Saved.


...

Generating Sub:   0%|          | 0/2 [00:00<?, ?it/s]


✅ submission_V6_multiobj.csv saved successfully.
Calculating final validation scores on full val set...


CSLS (Pass 1/2):   0%|          | 0/49 [00:00<?, ?it/s]

CSLS (Pass 2/2):   0%|          | 0/13 [00:00<?, ?it/s]


--- Full Validation Results ---
Validation Recall@1:  0.1894  
Validation Recall@5:  0.3961  
Validation Recall@10: 0.5094  
Validation Recall@50: 0.7691  
